# Bachelor. Conveyor belt state classification using deep neural networks and data augmentation methods.

Vilnius University \\
Software Engineering \\
Student Armantas Pikšrys

## Import and Mount data

In [ ]:
!pip install torchvision

In [ ]:
!pip install sktime
!pip install pmdarima
!pip install tsai

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sktime
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
import seaborn as sns
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.sarimax import SARIMAX
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.compose import (
    EnsembleForecaster,
    MultiplexForecaster,
    TransformedTargetForecaster,
    make_reduction,
)
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.performance_metrics.forecasting import  MeanAbsolutePercentageError, MeanSquaredError
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.transformations.panel.rocket import MiniRocket
from sktime.utils.plotting import plot_series
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor
from sktime.forecasting.fbprophet import Prophet
from sktime.forecasting.tbats import TBATS
from sktime.forecasting.ets import AutoETS
from sktime.performance_metrics.forecasting import  MeanAbsolutePercentageError, MeanSquaredError
from sktime.utils.plotting import plot_series
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sktime.datasets import load_from_tsfile
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
from tsai.all import *
from tsai.data.transforms import ToTensor
import warnings

computer_setup()

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print(gpu_devices)
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras import backend as K
from keras.models import Sequential
from keras.layers import TimeDistributed, Conv1D, BatchNormalization, AveragePooling1D, Dropout, Flatten
from keras.layers import LSTM, Dense
from keras import layers
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization, TimeDistributed, Conv1D, MaxPooling1D, Flatten
import tensorflow.keras as keras

## Read data

### Read weight data

In [ ]:
# Set the window size
window_size = 800

# Directory where your files are located
data_dir = "./drive/MyDrive/kursinis/WeightAndIncisionDataCleaned"

In [ ]:
# Initialize empty lists for signals and labels
signals = []
labels = []

In [ ]:
# Read Weight Data
# Iterate over files in the directory
for filename in os.listdir(data_dir):
    if filename.endswith(".csv"):
        file_path = os.path.join(data_dir, filename)
        if float(filename.split("_")[0]) == 0:
            weight_label = float(0.5)
        else:
            weight_label = float(filename.split("_")[0])

        # Load CSV data
        data = pd.read_csv(file_path)

        # Create signals using a sliding window
        print(filename)
        num_rows = len(data)
        step_size = 800

        for i in range(0, num_rows - window_size + 1, step_size):
            window_data = data.iloc[i:i+window_size]
            signals.append(window_data.values)
            labels.append(weight_label)

## Convert data

### Convert data regular

In [ ]:
# Convert the lists to NumPy arrays
signals_array = np.array(signals)
labels_array = np.array(labels)

In [ ]:
label_encoder = LabelEncoder()
labels_array = label_encoder.fit_transform(labels_array)

In [ ]:
unique_labels, counts = np.unique(labels_array, return_counts=True)
print("Unique Labels:", unique_labels)
print("Counts:", counts)
print(signals_array.shape)

### Signal mean calculation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
overall_mean_signal = np.mean(signals_array, axis=0)

In [ ]:
mean_signals_by_weight = []

for weight in [0.5,1,2,3,5,6]:
    mask = (labels_array == weight)
    mean_signal_weight = np.mean(signals_array[mask], axis=0)
    mean_signals_by_weight.append(mean_signal_weight)

mean_signals_by_weight = np.array(mean_signals_by_weight)

In [ ]:
import matplotlib.pyplot as plt

# Plotting
fig, axs = plt.subplots(1, 3, figsize=(13, 5), sharex=True)

# Convert time axis to seconds
time_seconds = np.arange(overall_mean_signal.shape[0]) / 400.0

# Define colors for each line
colors = ['C0', 'C1', 'C2', 'C3', 'C5', 'C5']

# Plot overall mean signal for each dimension
legend_entries = []

for i in range(3):
    # axs[i].plot(overall_mean_signal[:, i], label=f'Overall Mean - Dim {i + 1}')
    axs[i].set_xlabel('Laikas, s')
    axs[i].set_ylabel('Amplitudė, ADU')

    # Add label "a)", "b)", or "c)" to each subplot
    axs[i].text(0.01, 0.99, f'{chr(97 + i)})', transform=axs[i].transAxes, va='top', ha='left', fontsize=12)

    # Plot mean signals by weight category for each dimension
    for j, mean_signal_weight in enumerate(mean_signals_by_weight):
        label = f'{["Pažeistas",1,2,3,5,"Pažeistas"][j]}' if j == 0 else f'{["Pažeistas","0,5",1,2,3,5][j]} kg apkrova'
        # line = axs[i].plot(time_seconds, mean_signal_weight[:, i], label=label)[0]
        line = axs[i].plot(time_seconds, mean_signal_weight[:, i], label=label, color=colors[j])[0]
        # Collect legend entries only once
        if i == 0:
            legend_entries.append(line)

# Adjust legend position
fig.legend(legend_entries, labels=[line.get_label() for line in legend_entries], loc='upper left', bbox_to_anchor=(0.94, 0.92))

plt.tight_layout(rect=[0, 0, 0.95, 0.95])  # Adjust layout to prevent clipping of titles
plt.show()

## Split data into train, test, val

### Split and normalize regular

In [ ]:
N = signals_array.shape[0]
N_train = int(N*0.7)
N_val = int(N*0.9)

N_train, N_val, N

In [ ]:
np.random.seed(seed=123)
arr = np.arange(N)
np.random.shuffle(arr)

arr.shape

In [ ]:
mu = np.mean(signals_array[arr[:N_train]])
va = np.std(signals_array[arr[:N_train]])

In [ ]:
print(mu)
print(va)

In [ ]:
x_train = (signals_array[arr[:N_train]] - mu)/va
x_val   = (signals_array[arr[N_train:N_val]] - mu)/va
x_test  = (signals_array[arr[N_val:]] - mu)/va

y_train = labels_array[arr[:N_train]]
y_val   = labels_array[arr[N_train:N_val]]
y_test  = labels_array[arr[N_val:]]

n_classes = len(np.unique(y_train))

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape, n_classes

In [ ]:
print(np.mean(x_train))
print(np.std(x_train))
print(np.mean(x_val))
print(np.std(x_val))
print(np.mean(x_test))
print(np.std(x_test))

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train
                                    )
class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

### Split and normalize TSAI lib

In [ ]:
## train, test, validation splits
splits = get_splits(labels_array,
                    n_splits=1,
                    valid_size=0.2,
                    test_size=0.1,
                    shuffle=True,
                    balance=False,
                    stratify=True,
                    random_state=43,
                    show_plot=True,
                    verbose=True)
splits

In [ ]:
# Extract indices from the splits
train_indices = splits[0]
val_indices = splits[1]
test_indices = splits[2]

In [ ]:
# Compute mean and standard deviation over the training set
mu = np.mean(signals_array[train_indices])
va = np.std(signals_array[train_indices])

In [ ]:
print(mu)
print(va)

In [ ]:
# Normalize each split using the statistics computed from the training set
x_train = (signals_array[train_indices] - mu) / va
x_val = (signals_array[val_indices] - mu) / va
x_test = (signals_array[test_indices] - mu) / va

# Assign labels to each split
y_train = labels_array[train_indices]
y_val = labels_array[val_indices]
y_test = labels_array[test_indices]

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

In [ ]:
x_train = signals_array[train_indices]
x_val = signals_array[val_indices]
x_test = signals_array[test_indices]

y_train = labels_array[train_indices]
y_val = labels_array[val_indices]
y_test = labels_array[test_indices]

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

In [ ]:
print(np.mean(x_train))
print(np.std(x_train))
print(np.mean(x_val))
print(np.std(x_val))
print(np.mean(x_test))
print(np.std(x_test))

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train
                                    )
class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

## Data augmentations

### Data augmentation methods

In [ ]:
# Function to add Laplace noise to a single signal
def add_laplace_noise(signal, scale=0.01):
    noise = np.random.laplace(scale=scale, size=signal.shape)
    return signal + noise

In [ ]:
def add_laplace_noise_to_array(signal_array_values, signal_array_targets, scale=0.01, augmentation_multiplier=1):
    combined_array = signal_array_values
    combined_targets = signal_array_targets

    for i in range(augmentation_multiplier):
        noisy_array = signal_array_values.copy()

        # Apply Laplace noise to each signal in the array
        for i in range(noisy_array.shape[0]):
            for j in range(noisy_array.shape[2]):
                noisy_array[i, :, j] = add_laplace_noise(noisy_array[i, :, j])

        # Append the Laplace-noised signals to x_train
        combined_array = np.vstack([combined_array, noisy_array])

        # Update labels accordingly
        targets_original = signal_array_targets.copy()
        combined_targets = np.concatenate([combined_targets, targets_original])

    # Shuffle the combined data (regular + noised)
    shuffle_indices = np.random.permutation(combined_array.shape[0])
    combined_array = combined_array[shuffle_indices, :]
    combined_targets = combined_targets[shuffle_indices]

    return combined_array, combined_targets

In [ ]:
# Function to add drift augmentation to a single signal
def add_drift(signal, scale=0.01):
    num_points = signal.shape[0]
    time = np.arange(num_points)
    drift = np.random.normal(scale=scale, size=num_points)
    return signal + drift

In [ ]:
def add_drift_to_array(signal_array_values, signal_array_targets, scale=0.01, augmentation_multiplier=1):
    combined_array = signal_array_values
    combined_targets = signal_array_targets

    for i in range(augmentation_multiplier):
        drifted_array = signal_array_values.copy()

        # Apply drift augmentation to each signal in the array
        for i in range(drifted_array.shape[0]):
            for j in range(drifted_array.shape[2]):
                drifted_array[i, :, j] = add_drift(drifted_array[i, :, j], scale=scale)

        # Append the drifted signals to x_train
        combined_array = np.vstack([combined_array, drifted_array])

        # Update labels accordingly
        targets_original = signal_array_targets.copy()
        combined_targets = np.concatenate([combined_targets, targets_original])

    # Shuffle the combined data (regular + drifted)
    shuffle_indices = np.random.permutation(combined_array.shape[0])
    combined_array = combined_array[shuffle_indices, :]
    combined_targets = combined_targets[shuffle_indices]

    return combined_array, combined_targets

In [ ]:
def add_uniform_noise(signal, scale=0.01):
    noise = np.random.uniform(low=-scale, high=scale, size=signal.shape)
    return signal + noise

In [ ]:
def add_uniform_noise_to_array(signal_array_values, signal_array_targets, scale=0.01, augmentation_multiplier=1):
    combined_array = signal_array_values
    combined_targets = signal_array_targets

    for i in range(augmentation_multiplier):
        uniformed_array = signal_array_values.copy()

        # Apply drift augmentation to each signal in the array
        for i in range(uniformed_array.shape[0]):
            uniformed_array[i] = add_uniform_noise(uniformed_array[i], scale=scale)

        # Append the drifted signals to x_train
        combined_array = np.vstack([combined_array, uniformed_array])

        # Update labels accordingly
        targets_original = signal_array_targets.copy()
        combined_targets = np.concatenate([combined_targets, targets_original])

    # Shuffle the combined data (regular + drifted)
    shuffle_indices = np.random.permutation(combined_array.shape[0])
    combined_array = combined_array[shuffle_indices, :]
    combined_targets = combined_targets[shuffle_indices]

    return combined_array, combined_targets

In [ ]:
# Function to add combined Laplace noise and drift augmentation to a single signal
def add_laplace_and_drift_augmentation(signal, laplace_scale=0.01, drift_scale=0.01):
    # Apply Laplace noise
    signal_with_laplace = add_laplace_noise(signal, scale=laplace_scale)

    # Apply drift augmentation to the signal with Laplace noise
    signal_with_combined_augmentation = add_drift(signal_with_laplace, scale=drift_scale)

    return signal_with_combined_augmentation

In [ ]:
def add_laplace_and_drift_augmentation_to_array(signal_array_values, signal_array_targets, laplace_scale=0.01, drift_scale=0.01, augmentation_multiplier=1):
    combined_array = signal_array_values
    combined_targets = signal_array_targets

    for _ in range(augmentation_multiplier):
        augmented_array = signal_array_values.copy()

        # Apply combined augmentation to each signal in the array
        for i in range(augmented_array.shape[0]):
            for j in range(augmented_array.shape[2]):
                augmented_array[i, :, j] = add_laplace_and_drift_augmentation(augmented_array[i, :, j], laplace_scale=laplace_scale, drift_scale=drift_scale)

        # Append the augmented signals to x_train
        combined_array = np.vstack([combined_array, augmented_array])

        # Update labels accordingly
        targets_original = signal_array_targets.copy()
        combined_targets = np.concatenate([combined_targets, targets_original])

    # Shuffle the combined data (regular + augmented)
    shuffle_indices = np.random.permutation(combined_array.shape[0])
    combined_array = combined_array[shuffle_indices, :]
    combined_targets = combined_targets[shuffle_indices]

    return combined_array, combined_targets

In [ ]:
# Function to add combined Laplace noise and uniform noise augmentation to a single signal
def add_laplace_and_uniform_augmentation(signal, laplace_scale=0.01, uniform_scale=0.01):
    # Apply Laplace noise
    signal_with_laplace = add_laplace_noise(signal, scale=laplace_scale)

    # Apply uniform noise augmentation to the signal with Laplace noise
    signal_with_combined_augmentation = add_uniform_noise(signal_with_laplace, scale=uniform_scale)

    return signal_with_combined_augmentation

In [ ]:
def add_laplace_and_uniform_augmentation_to_array(signal_array_values, signal_array_targets, laplace_scale=0.01, uniform_scale=0.01, augmentation_multiplier=1):
    combined_array = signal_array_values
    combined_targets = signal_array_targets

    for _ in range(augmentation_multiplier):
        augmented_array = signal_array_values.copy()

        # Apply drift augmentation to each signal in the array
        for i in range(augmented_array.shape[0]):
            augmented_array[i] = add_laplace_and_uniform_augmentation(augmented_array[i], laplace_scale=laplace_scale, uniform_scale=uniform_scale)

        # Append the augmented signals to x_train
        combined_array = np.vstack([combined_array, augmented_array])

        # Update labels accordingly
        targets_original = signal_array_targets.copy()
        combined_targets = np.concatenate([combined_targets, targets_original])

    # Shuffle the combined data (regular + augmented)
    shuffle_indices = np.random.permutation(combined_array.shape[0])
    combined_array = combined_array[shuffle_indices, :]
    combined_targets = combined_targets[shuffle_indices]

    return combined_array, combined_targets

In [ ]:
from itertools import permutations

def all_channel_permutations_augmentation(data, labels):
    num_samples, width, num_channels = data.shape

    # Generate all possible channel permutations
    all_permutations = list(permutations(range(num_channels)))
    print(all_permutations)

    # Create arrays to store the augmented data and labels
    augmented_data = np.zeros((num_samples * len(all_permutations), width, num_channels))
    augmented_labels = np.zeros((num_samples * len(all_permutations),))

    # Iterate over each sample
    for i in range(num_samples):
        # Get the original channels and label for the current sample
        original_channels = data[i, :, :]
        original_label = labels[i]

        # Create all permutations for the current sample
        permuted_samples = [original_channels[:, perm] for perm in all_permutations]

        # Concatenate the permuted samples to the augmented data
        augmented_data[i * len(all_permutations): (i + 1) * len(all_permutations), :, :] = permuted_samples

        # Set the same label for all augmented samples
        augmented_labels[i * len(all_permutations): (i + 1) * len(all_permutations)] = original_label

    # Shuffle the augmented data and labels together
    shuffle_indices = np.random.permutation(len(augmented_labels))
    augmented_data = augmented_data[shuffle_indices, :, :]
    augmented_labels = augmented_labels[shuffle_indices]

    return augmented_data, augmented_labels

In [ ]:
def multiply_and_reshuffle(train_values, train_labels, multiplier=29):

    # Check if the multiplier is a positive integer
    if not isinstance(multiplier, int) or multiplier <= 0:
        raise ValueError("Multiplier should be a positive integer.")

    # Initialize arrays to store multiplied and reshuffled data
    multiplied_and_shuffled_values = train_values.copy()
    multiplied_and_shuffled_labels = train_labels.copy()

    for _ in range(multiplier - 1):
        # Multiply the data and labels
        multiplied_and_shuffled_values = np.vstack([multiplied_and_shuffled_values, train_values])
        multiplied_and_shuffled_labels = np.concatenate([multiplied_and_shuffled_labels, train_labels])

    # Shuffle the combined data
    shuffle_indices = np.random.permutation(multiplied_and_shuffled_values.shape[0])
    multiplied_and_shuffled_values = multiplied_and_shuffled_values[shuffle_indices, :]
    multiplied_and_shuffled_labels = multiplied_and_shuffled_labels[shuffle_indices]

    return multiplied_and_shuffled_values, multiplied_and_shuffled_labels


In [ ]:
def magnitude_warp(x, sigma=0.2, knot=4):
    from scipy.interpolate import CubicSpline
    orig_steps = np.arange(x.shape[1])

    random_warps = np.random.normal(loc=1.0, scale=sigma, size=(x.shape[0], knot+2, x.shape[2]))
    warp_steps = (np.ones((x.shape[2],1))*(np.linspace(0, x.shape[1]-1., num=knot+2))).T
    ret = np.zeros_like(x)
    for i, pat in enumerate(x):
        warper = np.array([CubicSpline(warp_steps[:,dim], random_warps[i,:,dim])(orig_steps) for dim in range(x.shape[2])]).T
        ret[i] = pat * warper

    return ret

In [ ]:
from scipy.interpolate import CubicSpline

def random_curve_generator(seq_len, dimensions, num_control_points=4):
    # Generate random points to construct the spline curve
    control_points = np.random.rand(num_control_points, dimensions) * 2 - 1  # Points in the range [-1, 1]
    # Ensure first and last points are zeros, so the output curve starts and ends at the original magnitude
    control_points[0], control_points[-1] = np.zeros(dimensions), np.zeros(dimensions)
    # Generate a sequence of X values (time) to interpolate
    x = np.linspace(0, 1, num_control_points)
    # Create a sequence of points to evaluate the spline
    x_eval = np.linspace(0, 1, seq_len)
    # Perform cubic spline interpolation
    cs = CubicSpline(x, control_points, axis=0)  # Interpolates along each dimension independently
    warp_curve = cs(x_eval)
    return warp_curve

def magnitude_warping(time_series, sigma=0.2):
    size, seq_len, dimensions = time_series.shape
    warped_time_series = np.empty_like(time_series)
    for i in range(size):
        # Obtain a random curve
        random_curve = random_curve_generator(seq_len, dimensions)
        # Perturb the time series using random curve
        warped_time_series[i] = time_series[i] * (1 + sigma * random_curve)
    return warped_time_series

In [ ]:
def add_magnitude_warping_to_array(signal_array_values, signal_array_targets, sigma=0.2, augmentation_multiplier=1):
    combined_array = signal_array_values
    combined_targets = signal_array_targets

    for i in range(augmentation_multiplier):
        warped_array = signal_array_values.copy()

        warped_array = magnitude_warping(warped_array, sigma)

        # Append the drifted signals to x_train
        combined_array = np.vstack([combined_array, warped_array])

        # Update labels accordingly
        targets_original = signal_array_targets.copy()
        combined_targets = np.concatenate([combined_targets, targets_original])

    # Shuffle the combined data (regular + drifted)
    shuffle_indices = np.random.permutation(combined_array.shape[0])
    combined_array = combined_array[shuffle_indices, :]
    combined_targets = combined_targets[shuffle_indices]

    return combined_array, combined_targets

### Applying data augmentations

In [ ]:
# x_train_noisy = x_train.copy()

# # Apply combined augmentation to each signal in the array
# for i in range(x_train.shape[0]):
#     for j in range(x_train.shape[2]):
#         x_train_noisy[i, :, j] = add_laplace_and_drift_augmentation(x_train[i, :, j], laplace_scale=0.01, drift_scale=0.01)

In [ ]:
# x_train_noisy, y_train_noisy = add_laplace_and_drift_augmentation_to_array(x_train, y_train, 0.01, 0.01, 62)

# x_train_noisy.shape, y_train_noisy.shape

In [ ]:
# x_train_noisy = x_train.copy()

# # Apply combined augmentation to each signal in the array
# for i in range(x_train.shape[0]):
#     x_train_noisy[i] = add_laplace_and_drift_augmentation(x_train_noisy[i], laplace_scale=0.01, uniform_scale=0.01)

In [ ]:
# x_train_noisy, y_train_noisy = add_laplace_and_uniform_augmentation_to_array(x_train, y_train, 0.01, 0.01, 62)

# x_train_noisy.shape, y_train_noisy.shape

In [ ]:
# x_train_noisy, y_train_noisy = add_magnitude_warping_to_array(x_train, y_train, 0.2, 30)

# x_train_noisy.shape, y_train_noisy.shape

In [ ]:
# x_train_noisy, y_train_noisy = add_uniform_noise_to_array(x_train, y_train, 0.02, 62)

# x_train_noisy.shape, y_train_noisy.shape

In [ ]:
# x_train_noisy, y_train_noisy = all_channel_permutations_augmentation(x_train, y_train)

# x_train_noisy.shape, y_train_noisy.shape

In [ ]:
x_train_noisy, y_train_noisy = add_laplace_noise_to_array(x_train, y_train, 0.01, 24)

x_train_noisy.shape, y_train_noisy.shape

In [ ]:
x_test_noisy, y_test_noisy = add_laplace_noise_to_array(x_test, y_test, 0.05, 100)

x_test_noisy.shape, y_test_noisy.shape

In [ ]:
# # Example usage
# x_train_noisy, y_train_noisy = add_drift_to_array(x_train, y_train, scale=0.01, augmentation_multiplier=40)

# x_train_noisy.shape, y_train_noisy.shape

In [ ]:
# x_test, y_test = add_laplace_noise_to_array(x_test, y_test, 0.02, 10)

# x_test.shape, y_test.shape

In [ ]:
# x_train_noisy, y_train_noisy = add_drift_to_array(x_train, y_train, 0.05, 100)

# x_train_noisy.shape, y_train_noisy.shape

In [ ]:
# x_val, y_val = add_laplace_noise_to_array(x_val, y_val, 0.01, 31)

# x_val.shape, y_val.shape

In [ ]:
# x_val, y_val = add_drift_to_array(x_val, y_val, 0.02, 10)

# x_val.shape, y_val.shape

In [ ]:
# Call the function
x_train_noisy, y_train_noisy = multiply_and_reshuffle(x_train, y_train, multiplier=1)

x_train_noisy.shape, y_train_noisy.shape

In [ ]:
# x_test_noisy, y_test_noisy = add_drift_to_array(x_test, y_test, 0.05, 100)

# x_test_noisy.shape, y_test_noisy.shape

In [ ]:
# x_train_noisy, y_train_noisy = multiply_and_reshuffle(x_train, y_train, multiplier=130)

# x_train_noisy.shape, y_train_noisy.shape

In [ ]:
# x_train_noisy, y_train_noisy = multiply_and_reshuffle(x_train_noisy, y_train_noisy, multiplier=5)

# x_train_noisy.shape, y_train_noisy.shape

### Show train data with augmentation

In [ ]:
import matplotlib.pyplot as plt

# Set plot style to ggplot
original = x_train[15]
# Define time axis
time_interval = 1 / 400  # Time interval in seconds per time point
time_axis = np.arange(window_size) * time_interval  # Convert time axis to seconds

# Plotting original and generated data side by side horizontally
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Plot original data
id1 = 4
axs[0].plot(time_axis,original[:, 0], label='Sensor 1')
axs[0].plot(time_axis, original[:, 1], label='Sensor 2')
axs[0].plot(time_axis, original[:, 2], label='Sensor 3')
axs[0].text(0.01, 0.99, f'{chr(97)})', transform=axs[0].transAxes, va='top', ha='left', fontsize=12)
axs[0].set_xlabel('Time (s)')
axs[0].set_ylabel('Amplitude Normalized (ADU)')
axs[0].legend()

# Plot generated data
id = 1
axs[1].plot(time_axis, original[:, 1], label='Sensor 1')
axs[1].plot(time_axis, original[:, 2], label='Sensor 2')
axs[1].plot(time_axis, original[:, 0], label='Sensor 3')
axs[1].text(0.01, 0.99, f'{chr(98)})', transform=axs[1].transAxes, va='top', ha='left', fontsize=12)
axs[1].set_xlabel('Time (s)')
axs[1].set_ylabel('Amplitude Normalized (ADU)')
axs[1].legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have signals_array containing your original signals
original = x_train[15]

# Generate Laplace noise
laplace_noise_3 = add_laplace_noise(original, 0.01)
laplace_noise_2 = add_laplace_noise(original, 0.02)
laplace_noise_1 = add_laplace_noise(original, 0.05)

# Convert time axis to seconds
time_seconds = np.arange(signals_array[0].shape[0]) / 400.0

# Plot all channels in a single plot
plt.figure(figsize=(12, 6))

plt.plot(time_seconds, original[:, 0], label='Original', color='blue')
plt.plot(time_seconds, original[:, 1], color='blue')
plt.plot(time_seconds, original[:, 2], color='blue')


plt.plot(time_seconds, laplace_noise_3[:, 0], label='Laplace std/100', color='green')
plt.plot(time_seconds, laplace_noise_3[:, 1], color='green')
plt.plot(time_seconds, laplace_noise_3[:, 2], color='green')


plt.plot(time_seconds, laplace_noise_2[:, 0], label='Laplace std/50', color='red')
plt.plot(time_seconds, laplace_noise_2[:, 1], color='red')
plt.plot(time_seconds, laplace_noise_2[:, 2], color='red')

plt.plot(time_seconds, laplace_noise_1[:, 0], label='Laplace std/20', color='orange')
plt.plot(time_seconds, laplace_noise_1[:, 1], color='orange')
plt.plot(time_seconds, laplace_noise_1[:, 2], color='orange')

plt.xlabel('Time (s)')
plt.ylabel('Amplitude Normalized (ADU)')
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have signals_array containing your original signals
original = x_train[15]

# Generate Laplace noise
warped = magnitude_warping(x_train[:20], 0.2)

# Convert time axis to seconds
time_seconds = np.arange(signals_array[0].shape[0]) / 400.0

# Plot all channels in a single plot
plt.figure(figsize=(12, 6))

plt.plot(time_seconds, original[:, 0], label='Original', color='blue')
plt.plot(time_seconds, original[:, 1], color='blue')
plt.plot(time_seconds, original[:, 2], color='blue')


plt.plot(time_seconds, warped[15][:, 0], label='Magnitude warping', color='green')
plt.plot(time_seconds, warped[15][:, 1], color='green')
plt.plot(time_seconds, warped[15][:, 2], color='green')


plt.xlabel('Time (s)')
plt.ylabel('Amplitude Normalized (ADU)')
plt.legend()

plt.show()

## Models

### Resnet

#### Resnet helpers

In [ ]:
# resnet model
# when tuning start with learning rate->mini_batch_size ->
# momentum-> #hidden_units -> # learning_rate_decay -> #layers
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time
import sklearn

import matplotlib

import matplotlib.pyplot as plt

In [ ]:
from builtins import print
import numpy as np
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt

import os
import operator

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import LabelEncoder

from scipy.interpolate import interp1d
from scipy.io import loadmat

def calculate_metrics(y_true, y_pred, duration, y_true_val=None, y_pred_val=None):
    res = pd.DataFrame(data=np.zeros((1, 4), dtype=float), index=[0],
                       columns=['precision', 'accuracy', 'recall', 'duration'])
    res['precision'] = precision_score(y_true, y_pred, average='macro')
    res['accuracy'] = accuracy_score(y_true, y_pred)

    if not y_true_val is None:
        # this is useful when transfer learning is used with cross validation
        res['accuracy_val'] = accuracy_score(y_true_val, y_pred_val)

    res['recall'] = recall_score(y_true, y_pred, average='macro')
    res['duration'] = duration
    return res


def save_test_duration(file_name, test_duration):
    res = pd.DataFrame(data=np.zeros((1, 1), dtype=float), index=[0],
                       columns=['test_duration'])
    res['test_duration'] = test_duration
    res.to_csv(file_name, index=False)


def plot_epochs_metric(hist, file_name, metric='loss'):
    plt.figure()
    plt.plot(hist.history[metric])
    plt.plot(hist.history['val_' + metric])
    plt.title('model ' + metric)
    plt.ylabel(metric, fontsize='large')
    plt.xlabel('epoch', fontsize='large')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(file_name, bbox_inches='tight')
    plt.close()


def save_logs_t_leNet(output_directory, hist, y_pred, y_true, duration):
    hist_df = pd.DataFrame(hist.history)
    hist_df.to_csv(output_directory + 'history.csv', index=False)

    df_metrics = calculate_metrics(y_true, y_pred, duration)
    df_metrics.to_csv(output_directory + 'df_metrics.csv', index=False)

    index_best_model = hist_df['loss'].idxmin()
    row_best_model = hist_df.loc[index_best_model]

    df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=float), index=[0],
                                 columns=['best_model_train_loss', 'best_model_val_loss', 'best_model_train_acc',
                                          'best_model_val_acc', 'best_model_learning_rate', 'best_model_nb_epoch'])

    df_best_model['best_model_train_loss'] = row_best_model['loss']
    df_best_model['best_model_val_loss'] = row_best_model['val_loss']
    df_best_model['best_model_train_acc'] = row_best_model['acc']
    df_best_model['best_model_val_acc'] = row_best_model['val_acc']
    df_best_model['best_model_nb_epoch'] = index_best_model

    df_best_model.to_csv(output_directory + 'df_best_model.csv', index=False)

    # plot losses
    plot_epochs_metric(hist, output_directory + 'epochs_loss.png')


def save_logs(output_directory, hist, y_pred, y_true, duration, lr=True, y_true_val=None, y_pred_val=None):
    hist_df = pd.DataFrame(hist.history)
    hist_df.to_csv(output_directory + 'history.csv', index=False)

    df_metrics = calculate_metrics(y_true, y_pred, duration, y_true_val, y_pred_val)
    df_metrics.to_csv(output_directory + 'df_metrics.csv', index=False)

    index_best_model = hist_df['loss'].idxmin()
    row_best_model = hist_df.loc[index_best_model]

    df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=float), index=[0],
                                 columns=['best_model_train_loss', 'best_model_val_loss', 'best_model_train_acc',
                                          'best_model_val_acc', 'best_model_learning_rate', 'best_model_nb_epoch'])

    df_best_model['best_model_train_loss'] = row_best_model['loss']
    df_best_model['best_model_val_loss'] = row_best_model['val_loss']
    df_best_model['best_model_train_acc'] = row_best_model['accuracy']
    df_best_model['best_model_val_acc'] = row_best_model['val_accuracy']
    if lr == True:
        df_best_model['best_model_learning_rate'] = row_best_model['lr']
    df_best_model['best_model_nb_epoch'] = index_best_model

    df_best_model.to_csv(output_directory + 'df_best_model.csv', index=False)

    # for FCN there is no hyperparameters fine tuning - everything is static in code

    # plot losses
    plot_epochs_metric(hist, output_directory + 'epochs_loss.png')

    return df_metrics

#### Resnet classifier Regular

In [ ]:
class Classifier_RESNET:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, load_weights=False):
        self.output_directory = output_directory
        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            if load_weights == True:
                self.model.load_weights(self.output_directory
                                        .replace('resnet_augment', 'resnet')
                                        .replace('TSC_itr_augment_x_10', 'TSC_itr_10')
                                        + '/model_init.weights.h5')
            else:
                self.model.save_weights(self.output_directory + 'model_init.weights.h5')
        return

    def build_model(self, input_shape, nb_classes):
        n_feature_maps = 64

        input_layer = keras.layers.Input(input_shape)

        # BLOCK 1

        conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_1 = keras.layers.add([shortcut_y, conv_z])
        output_block_1 = keras.layers.Activation('relu')(output_block_1)

        # BLOCK 2

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # expand channels for the sum
        shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        output_block_2 = keras.layers.add([shortcut_y, conv_z])
        output_block_2 = keras.layers.Activation('relu')(output_block_2)

        # BLOCK 3

        conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
        conv_x = keras.layers.BatchNormalization()(conv_x)
        conv_x = keras.layers.Activation('relu')(conv_x)

        conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
        conv_y = keras.layers.BatchNormalization()(conv_y)
        conv_y = keras.layers.Activation('relu')(conv_y)

        conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
        conv_z = keras.layers.BatchNormalization()(conv_z)

        # no need to expand channels because they are equal
        shortcut_y = keras.layers.BatchNormalization()(output_block_2)

        output_block_3 = keras.layers.add([shortcut_y, conv_z])
        output_block_3 = keras.layers.Activation('relu')(output_block_3)

        # FINAL

        gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)

#         file_path = self.output_directory + 'best_model.{epoch:02d}-{val_loss:.2f}.weights.h5'

#         model_checkpoint = keras.callbacks.ModelCheckpoint(
#             filepath=file_path,
#             monitor='val_accuracy',
#             mode='max',
#             verbose=1,
#             save_best_only=True)

        self.callbacks = [reduce_lr]

        return model

    def fit(self, x_train, y_train, x_val, y_val, y_true, batch_size, nb_epochs, shuffle, callbacks, class_weights):
        if not tf.test.is_gpu_available:
            print('error')
            exit()

        mini_batch_size = int(min(x_train.shape[0] / 10, batch_size))

        start_time = time.time()

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                              verbose=1, validation_data=(x_val, y_val), callbacks=callbacks, shuffle=shuffle, class_weight=class_weights)

        # duration = time.time() - start_time

        # self.model.save(self.output_directory + 'last_model.h5')

        # y_pred = self.predict(x_val, y_true, x_train, y_train, y_val,
        #                       return_df_metrics=False)

        # # save predictions
        # np.save(self.output_directory + 'y_pred.npy', y_pred)

        # # convert the predicted from binary to integer
        # y_pred = np.argmax(y_pred, axis=1)

        # df_metrics = save_logs(self.output_directory, hist, y_pred, y_true, duration)

        # keras.backend.clear_session()

        # return df_metrics

    def predict(self, x_test, y_true, x_train, y_train, y_test, return_df_metrics=True):
        start_time = time.time()
        model_path = self.output_directory + 'best_model.weights.h5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            test_duration = time.time() - start_time
            save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
            return y_pred

### CNN-LSTM Original

In [ ]:
## Long short-term memory CNN-LSTM ###
def lstm(n_timesteps, n_features, n_steps,  n_outputs):
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=512, kernel_size=3, activation='relu'), input_shape=(n_steps, n_timesteps // n_steps, n_features)))

    model.add(BatchNormalization())
    model.add(TimeDistributed(AveragePooling1D(pool_size=2)))
    model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))

    model.add(BatchNormalization())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(AveragePooling1D(pool_size=2)))
    model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))

    model.add(BatchNormalization())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(AveragePooling1D(pool_size=2)))
    model.add(TimeDistributed(Conv1D(filters=8, kernel_size=3, activation='relu')))

    model.add(BatchNormalization())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(AveragePooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))

    model.add(layers.LSTM(16, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(layers.LSTM(16))
    model.add(Dense(n_outputs, activation='softmax'))
    return model

### Regular LSTM

In [ ]:
# Build the Regular study LSTM model
model = Sequential()
model.add(LSTM(units=128, input_shape=(window_size, data.shape[1]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=8))
model.add(Dense(units=len(unique_labels), activation='softmax'))  # Adjust the number of units based on your task

### FCN (Fully convolutional network)

In [ ]:
def fcn(input_shape, num_classes):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

### Inception

In [ ]:
class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, batch_size=64, lr=0.001,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.verbose = verbose

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > self.bottleneck_size:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(self.lr),
                      metrics=['accuracy'])

        # reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=20,
        #                                               min_lr=0.0001)

        # file_path = self.output_directory + 'best_model.{epoch:02d}-{val_loss:.2f}.h5'

        # model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='val_accuracy',
        #                                                    save_best_only=True, verbose=1)

        # self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit(self, x_train, y_train, x_val, y_val, y_true, batch_size, nb_epochs, shuffle, callbacks, class_weights):
        if not tf.test.is_gpu_available:
            print('error no gpu')
            exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training

        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 10, 16))
        else:
            mini_batch_size = self.batch_size

        start_time = time.time()

        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                              verbose=self.verbose, validation_data=(x_val, y_val), callbacks=callbacks, shuffle=shuffle, class_weight=class_weights)

        # duration = time.time() - start_time

        # self.model.save(self.output_directory + 'last_model.hdf5')

        # y_pred = self.predict(x_val, y_true, x_train, y_train, y_val,
        #                       return_df_metrics=False)

        # # save predictions
        # np.save(self.output_directory + 'y_pred.npy', y_pred)

        # # convert the predicted from binary to integer
        # y_pred = np.argmax(y_pred, axis=1)

        # df_metrics = save_logs(self.output_directory, hist, y_pred, y_true, duration)

        # keras.backend.clear_session()

        # return df_metrics

    def predict(self, x_test, y_true, x_train, y_train, y_test, return_df_metrics=True):
        start_time = time.time()
        model_path = self.output_directory + 'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test, batch_size=self.batch_size)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            test_duration = time.time() - start_time
            save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
            return y_pred

### MLSTM-FCN

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout
from keras import backend as K

def mlstm_fcn(input_shape, num_classes):
    ip = Input(input_shape)

    x = Masking()(ip)
    x = LSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)
    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(num_classes, activation='softmax')(x)

    model = Model(ip, out)
    # add load model code here to fine-tune

    return model

def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor

    Returns: a keras tensor
    '''
    # filters = tf.shape(input)[-1] # channel_axis = -1 for TF
    filters = K.int_shape(input)[-1]

    se = GlobalAveragePooling1D()(input)
    # se = Reshape((1, filters))(se)
    se = K.reshape(se, (-1, 1, filters))
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se

### MultiRocket

#### Helpers

In [ ]:
import copy

import numpy as np
import torch
import torch.nn.functional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm


class LogisticRegression:

    def __init__(
            self,
            num_features,
            max_epochs=500,
            minibatch_size=256,
            validation_size=2 ** 11,
            learning_rate=1e-4,
            patience_lr=5,  # 500 minibatches
            patience=10,  # 1000 minibatches
            device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ):
        self.name = "LogisticRegression"
        self.args = {
            "num_features": num_features,
            "validation_size": validation_size,
            "minibatch_size": minibatch_size,
            "lr": learning_rate,
            "max_epochs": max_epochs,
            "patience_lr": patience_lr,
            "patience": patience,
        }

        self.model = None
        self.device = device
        self.classes = None
        self.scaler = None
        self.num_classes = None

    def fit(self, x_train, y_train):
        self.classes = np.unique(y_train)
        self.num_classes = len(self.classes)

        num_outputs = self.num_classes if self.num_classes > 2 else 1
        train_steps = int(x_train.shape[0] / self.args["minibatch_size"])

        self.scaler = StandardScaler()
        x_train = self.scaler.fit_transform(x_train)

        model = torch.nn.Sequential(torch.nn.Linear(self.args["num_features"], num_outputs)).to(self.device)

        if num_outputs == 1:
            loss_function = torch.nn.BCEWithLogitsLoss()
        else:
            loss_function = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=self.args["lr"])
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            factor=0.5,
            min_lr=1e-8,
            patience=self.args["patience_lr"]
        )

        training_size = x_train.shape[0]
        if self.args["validation_size"] < training_size:
            x_training, x_validation, y_training, y_validation = train_test_split(
                x_train, y_train,
                test_size=self.args["validation_size"],
                stratify=y_train
            )

            train_data = TensorDataset(
                torch.tensor(x_training, dtype=torch.float32).to(self.device),
                torch.tensor(y_training, dtype=torch.long).to(self.device)
            )
            val_data = TensorDataset(
                torch.tensor(x_validation, dtype=torch.float32).to(self.device),
                torch.tensor(y_validation, dtype=torch.long).to(self.device)
            )
            train_dataloader = DataLoader(train_data, shuffle=True, batch_size=self.args["minibatch_size"])
            val_dataloader = DataLoader(val_data, batch_size=self.args["minibatch_size"])
        else:
            train_data = TensorDataset(
                torch.tensor(x_train, dtype=torch.float32).to(self.device),
                torch.tensor(y_train, dtype=torch.long).to(self.device)
            )
            train_dataloader = DataLoader(train_data, shuffle=True, batch_size=self.args["minibatch_size"])
            val_dataloader = None

        best_loss = np.inf
        best_model = None
        stall_count = 0
        stop = False

        for epoch in range(self.args["max_epochs"]):
            if epoch > 0 and stop:
                break
            model.train()

            # loop over the training set
            total_train_loss = 0
            steps = 0
            for i, data in tqdm(enumerate(train_dataloader), desc=f"epoch: {epoch}", total=train_steps):
                x, y = data

                y_hat = model(x)
                if num_outputs == 1:
                    loss = loss_function(y_hat.sigmoid(), y)
                else:
                    yhat = torch.nn.functional.softmax(y_hat, dim=1)
                    loss = loss_function(yhat, y)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_train_loss += loss
                steps += 1

            total_train_loss = total_train_loss.cpu().detach().numpy() / steps

            if val_dataloader is not None:
                total_val_loss = 0
                # switch off autograd for evaluation
                with torch.no_grad():
                    # set the model in evaluation mode
                    model.eval()
                    for i, data in enumerate(val_dataloader):
                        x, y = data

                        y_hat = model(x)
                        if num_outputs == 1:
                            total_val_loss += loss_function(y_hat.sigmoid(), y)
                        else:
                            yhat = torch.nn.functional.softmax(y_hat, dim=1)
                            total_val_loss += loss_function(yhat, y)
                total_val_loss = total_val_loss.cpu().detach().numpy() / steps
                scheduler.step(total_val_loss)

                if total_val_loss >= best_loss:
                    stall_count += 1
                    if stall_count >= self.args["patience"]:
                        stop = True
                        print(f"\n<Stopped at Epoch {epoch + 1}>")
                else:
                    best_loss = total_val_loss
                    best_model = copy.deepcopy(model)
                    if not stop:
                        stall_count = 0
            else:
                scheduler.step(total_train_loss)
                if total_train_loss >= best_loss:
                    stall_count += 1
                    if stall_count >= self.args["patience"]:
                        stop = True
                        print(f"\n<Stopped at Epoch {epoch + 1}>")
                else:
                    best_loss = total_train_loss
                    best_model = copy.deepcopy(model)
                    if not stop:
                        stall_count = 0

        self.model = best_model
        return self.model

    def predict(self, x):
        x = self.scaler.transform(x)

        with torch.no_grad():
            # set the model in evaluation mode
            self.model.eval()

            yhat = self.model(torch.tensor(x, dtype=torch.float32).to(self.device))

            if self.num_classes > 2:
                yhat = self.classes[np.argmax(yhat.cpu().detach().numpy(), axis=1)]
            else:
                yhat = torch.sigmoid(yhat)
                yhat = np.round(yhat.cpu().detach().numpy())

            return yhat

In [ ]:
def fill_missing(x: np.array,
                 max_len: int,
                 vary_len: str = "suffix-noise",
                 normalise: bool = True):
    if vary_len == "zero":
        if normalise:
            x = StandardScaler().fit_transform(x)
        x = np.nan_to_num(x)
    elif vary_len == 'prefix-suffix-noise':
        for i in range(len(x)):
            series = list()
            for a in x[i, :]:
                if np.isnan(a):
                    break
                series.append(a)
            series = np.array(series)
            seq_len = len(series)
            diff_len = int(0.5 * (max_len - seq_len))

            for j in range(diff_len):
                x[i, j] = random.random() / 1000

            for j in range(diff_len, seq_len):
                x[i, j] = series[j - seq_len]

            for j in range(seq_len, max_len):
                x[i, j] = random.random() / 1000

            if normalise:
                tmp = StandardScaler().fit_transform(x[i].reshape(-1, 1))
                x[i] = tmp[:, 0]
    elif vary_len == 'uniform-scaling':
        for i in range(len(x)):
            series = list()
            for a in x[i, :]:
                if np.isnan(a):
                    break
                series.append(a)
            series = np.array(series)
            seq_len = len(series)

            for j in range(max_len):
                scaling_factor = int(j * seq_len / max_len)
                x[i, j] = series[scaling_factor]
            if normalise:
                tmp = StandardScaler().fit_transform(x[i].reshape(-1, 1))
                x[i] = tmp[:, 0]
    else:
        for i in range(len(x)):
            for j in range(len(x[i])):
                if np.isnan(x[i, j]):
                    x[i, j] = random.random() / 1000

            if normalise:
                tmp = StandardScaler().fit_transform(x[i].reshape(-1, 1))
                x[i] = tmp[:, 0]

    return x


In [ ]:
def process_ts_data(X,
                    vary_len: str = "suffix-noise",
                    normalise: bool = False):
    """
    This is a function to process the data, i.e. convert dataframe to numpy array
    :param X:
    :param normalise:
    :return:
    """
    num_instances, num_dim = X.shape
    columns = X.columns
    max_len = np.max([len(X[columns[0]][i]) for i in range(num_instances)])
    output = np.zeros((num_instances, num_dim, max_len), dtype=np.float64)

    for i in range(num_dim):
        for j in range(num_instances):
            output[j, i, :] = X[columns[i]][j].values
        output[:, i, :] = fill_missing(
            output[:, i, :],
            max_len,
            vary_len,
            normalise
        )

    return output

#### Architecture

In [ ]:
# Chang Wei Tan, Angus Dempster, Christoph Bergmeir, Geoffrey I Webb
#
# MultiRocket: Multiple pooling operators and transformations for fast and effective time series classification
# https://arxiv.org/abs/2102.00457

import time

import numpy as np
from numba import njit, prange
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


@njit("float32[:](float64[:,:,:],int32[:],int32[:],int32[:],int32[:],float32[:])",
      fastmath=True, parallel=False, cache=True)
def _fit_biases(X, num_channels_per_combination, channel_indices, dilations, num_features_per_dilation, quantiles):
    num_examples, num_channels, input_length = X.shape

    # equivalent to:
    # >>> from itertools import combinations
    # >>> indices = np.array([_ for _ in combinations(np.arange(9), 3)], dtype = np.int32)
    indices = np.array((
        0, 1, 2, 0, 1, 3, 0, 1, 4, 0, 1, 5, 0, 1, 6, 0, 1, 7, 0, 1, 8,
        0, 2, 3, 0, 2, 4, 0, 2, 5, 0, 2, 6, 0, 2, 7, 0, 2, 8, 0, 3, 4,
        0, 3, 5, 0, 3, 6, 0, 3, 7, 0, 3, 8, 0, 4, 5, 0, 4, 6, 0, 4, 7,
        0, 4, 8, 0, 5, 6, 0, 5, 7, 0, 5, 8, 0, 6, 7, 0, 6, 8, 0, 7, 8,
        1, 2, 3, 1, 2, 4, 1, 2, 5, 1, 2, 6, 1, 2, 7, 1, 2, 8, 1, 3, 4,
        1, 3, 5, 1, 3, 6, 1, 3, 7, 1, 3, 8, 1, 4, 5, 1, 4, 6, 1, 4, 7,
        1, 4, 8, 1, 5, 6, 1, 5, 7, 1, 5, 8, 1, 6, 7, 1, 6, 8, 1, 7, 8,
        2, 3, 4, 2, 3, 5, 2, 3, 6, 2, 3, 7, 2, 3, 8, 2, 4, 5, 2, 4, 6,
        2, 4, 7, 2, 4, 8, 2, 5, 6, 2, 5, 7, 2, 5, 8, 2, 6, 7, 2, 6, 8,
        2, 7, 8, 3, 4, 5, 3, 4, 6, 3, 4, 7, 3, 4, 8, 3, 5, 6, 3, 5, 7,
        3, 5, 8, 3, 6, 7, 3, 6, 8, 3, 7, 8, 4, 5, 6, 4, 5, 7, 4, 5, 8,
        4, 6, 7, 4, 6, 8, 4, 7, 8, 5, 6, 7, 5, 6, 8, 5, 7, 8, 6, 7, 8
    ), dtype=np.int32).reshape(84, 3)

    num_kernels = len(indices)
    num_dilations = len(dilations)

    num_features = num_kernels * np.sum(num_features_per_dilation)

    biases = np.zeros(num_features, dtype=np.float32)

    feature_index_start = 0

    combination_index = 0
    num_channels_start = 0

    for dilation_index in range(num_dilations):

        dilation = dilations[dilation_index]
        padding = ((9 - 1) * dilation) // 2

        num_features_this_dilation = num_features_per_dilation[dilation_index]

        for kernel_index in range(num_kernels):

            feature_index_end = feature_index_start + num_features_this_dilation

            num_channels_this_combination = num_channels_per_combination[combination_index]

            num_channels_end = num_channels_start + num_channels_this_combination

            channels_this_combination = channel_indices[num_channels_start:num_channels_end]

            _X = X[np.random.randint(num_examples)][channels_this_combination]

            A = -_X  # A = alpha * X = -X
            G = _X + _X + _X  # G = gamma * X = 3X

            C_alpha = np.zeros((num_channels_this_combination, input_length), dtype=np.float32)
            C_alpha[:] = A

            C_gamma = np.zeros((9, num_channels_this_combination, input_length), dtype=np.float32)
            C_gamma[9 // 2] = G

            start = dilation
            end = input_length - padding

            for gamma_index in range(9 // 2):
                C_alpha[:, -end:] = C_alpha[:, -end:] + A[:, :end]
                C_gamma[gamma_index, :, -end:] = G[:, :end]

                end += dilation

            for gamma_index in range(9 // 2 + 1, 9):
                C_alpha[:, :-start] = C_alpha[:, :-start] + A[:, start:]
                C_gamma[gamma_index, :, :-start] = G[:, start:]

                start += dilation

            index_0, index_1, index_2 = indices[kernel_index]

            C = C_alpha + C_gamma[index_0] + C_gamma[index_1] + C_gamma[index_2]
            C = np.sum(C, axis=0)

            biases[feature_index_start:feature_index_end] = np.quantile(C, quantiles[
                                                                           feature_index_start:feature_index_end])

            feature_index_start = feature_index_end

            combination_index += 1
            num_channels_start = num_channels_end

    return biases


def _fit_dilations(input_length, num_features, max_dilations_per_kernel):
    num_kernels = 84

    num_features_per_kernel = num_features // num_kernels
    true_max_dilations_per_kernel = min(num_features_per_kernel, max_dilations_per_kernel)
    multiplier = num_features_per_kernel / true_max_dilations_per_kernel

    print("TEST3")
    max_exponent = np.log2((input_length - 1) / (9 - 1))
    dilations, num_features_per_dilation = \
        np.unique(np.logspace(0, max_exponent, true_max_dilations_per_kernel, base=2).astype(np.int32),
                  return_counts=True)
    num_features_per_dilation = (num_features_per_dilation * multiplier).astype(np.int32)  # this is a vector

    remainder = num_features_per_kernel - np.sum(num_features_per_dilation)
    i = 0
    while remainder > 0:
        num_features_per_dilation[i] += 1
        remainder -= 1
        i = (i + 1) % len(num_features_per_dilation)

    return dilations, num_features_per_dilation


# low-discrepancy sequence to assign quantiles to kernel/dilation combinations
def _quantiles(n):
    return np.array([(_ * ((np.sqrt(5) + 1) / 2)) % 1 for _ in range(1, n + 1)], dtype=np.float32)


def fit(X, num_features=10_000, max_dilations_per_kernel=32):
    _, num_channels, input_length = X.shape

    num_kernels = 84

    dilations, num_features_per_dilation = _fit_dilations(input_length, num_features, max_dilations_per_kernel)

    num_features_per_kernel = np.sum(num_features_per_dilation)

    quantiles = _quantiles(num_kernels * num_features_per_kernel)

    num_dilations = len(dilations)
    num_combinations = num_kernels * num_dilations

    max_num_channels = min(num_channels, 9)
    max_exponent = np.log2(max_num_channels + 1)

    num_channels_per_combination = (2 ** np.random.uniform(0, max_exponent, num_combinations)).astype(np.int32)

    channel_indices = np.zeros(num_channels_per_combination.sum(), dtype=np.int32)

    num_channels_start = 0
    for combination_index in range(num_combinations):
        num_channels_this_combination = num_channels_per_combination[combination_index]
        num_channels_end = num_channels_start + num_channels_this_combination
        channel_indices[num_channels_start:num_channels_end] = np.random.choice(num_channels,
                                                                                num_channels_this_combination,
                                                                                replace=False)

        num_channels_start = num_channels_end

    biases = _fit_biases(X, num_channels_per_combination, channel_indices,
                         dilations, num_features_per_dilation, quantiles)

    return num_channels_per_combination, channel_indices, dilations, num_features_per_dilation, biases


@njit(
    "float32[:,:](float64[:,:,:],float64[:,:,:],Tuple((int32[:],int32[:],int32[:],int32[:],float32[:])),Tuple((int32[:],int32[:],int32[:],int32[:],float32[:])),int32)",
    fastmath=True, parallel=True, cache=True)
def transform(X, X1, parameters, parameters1, n_features_per_kernel=4):
    num_examples, num_channels, input_length = X.shape

    num_channels_per_combination, channel_indices, dilations, num_features_per_dilation, biases = parameters
    _, _, dilations1, num_features_per_dilation1, biases1 = parameters1

    # equivalent to:
    # >>> from itertools import combinations
    # >>> indices = np.array([_ for _ in combinations(np.arange(9), 3)], dtype = np.int32)
    indices = np.array((
        0, 1, 2, 0, 1, 3, 0, 1, 4, 0, 1, 5, 0, 1, 6, 0, 1, 7, 0, 1, 8,
        0, 2, 3, 0, 2, 4, 0, 2, 5, 0, 2, 6, 0, 2, 7, 0, 2, 8, 0, 3, 4,
        0, 3, 5, 0, 3, 6, 0, 3, 7, 0, 3, 8, 0, 4, 5, 0, 4, 6, 0, 4, 7,
        0, 4, 8, 0, 5, 6, 0, 5, 7, 0, 5, 8, 0, 6, 7, 0, 6, 8, 0, 7, 8,
        1, 2, 3, 1, 2, 4, 1, 2, 5, 1, 2, 6, 1, 2, 7, 1, 2, 8, 1, 3, 4,
        1, 3, 5, 1, 3, 6, 1, 3, 7, 1, 3, 8, 1, 4, 5, 1, 4, 6, 1, 4, 7,
        1, 4, 8, 1, 5, 6, 1, 5, 7, 1, 5, 8, 1, 6, 7, 1, 6, 8, 1, 7, 8,
        2, 3, 4, 2, 3, 5, 2, 3, 6, 2, 3, 7, 2, 3, 8, 2, 4, 5, 2, 4, 6,
        2, 4, 7, 2, 4, 8, 2, 5, 6, 2, 5, 7, 2, 5, 8, 2, 6, 7, 2, 6, 8,
        2, 7, 8, 3, 4, 5, 3, 4, 6, 3, 4, 7, 3, 4, 8, 3, 5, 6, 3, 5, 7,
        3, 5, 8, 3, 6, 7, 3, 6, 8, 3, 7, 8, 4, 5, 6, 4, 5, 7, 4, 5, 8,
        4, 6, 7, 4, 6, 8, 4, 7, 8, 5, 6, 7, 5, 6, 8, 5, 7, 8, 6, 7, 8
    ), dtype=np.int32).reshape(84, 3)

    num_kernels = len(indices)
    num_dilations = len(dilations)
    num_dilations1 = len(dilations1)

    num_features = num_kernels * np.sum(num_features_per_dilation)
    num_features1 = num_kernels * np.sum(num_features_per_dilation1)

    features = np.zeros((num_examples, (num_features + num_features1) * n_features_per_kernel), dtype=np.float32)
    n_features_per_transform = np.int64(features.shape[1] / 2)

    for example_index in prange(num_examples):

        _X = X[example_index]

        A = -_X  # A = alpha * X = -X
        G = _X + _X + _X  # G = gamma * X = 3X

        # Base series
        feature_index_start = 0

        combination_index = 0
        num_channels_start = 0

        for dilation_index in range(num_dilations):

            _padding0 = dilation_index % 2

            dilation = dilations[dilation_index]
            padding = ((9 - 1) * dilation) // 2

            num_features_this_dilation = num_features_per_dilation[dilation_index]

            C_alpha = np.zeros((num_channels, input_length), dtype=np.float32)
            C_alpha[:] = A

            C_gamma = np.zeros((9, num_channels, input_length), dtype=np.float32)
            C_gamma[9 // 2] = G

            start = dilation
            end = input_length - padding

            for gamma_index in range(9 // 2):
                C_alpha[:, -end:] = C_alpha[:, -end:] + A[:, :end]
                C_gamma[gamma_index, :, -end:] = G[:, :end]

                end += dilation

            for gamma_index in range(9 // 2 + 1, 9):
                C_alpha[:, :-start] = C_alpha[:, :-start] + A[:, start:]
                C_gamma[gamma_index, :, :-start] = G[:, start:]

                start += dilation

            for kernel_index in range(num_kernels):

                feature_index_end = feature_index_start + num_features_this_dilation

                num_channels_this_combination = num_channels_per_combination[combination_index]

                num_channels_end = num_channels_start + num_channels_this_combination

                channels_this_combination = channel_indices[num_channels_start:num_channels_end]

                _padding1 = (_padding0 + kernel_index) % 2

                index_0, index_1, index_2 = indices[kernel_index]

                C = C_alpha[channels_this_combination] + \
                    C_gamma[index_0][channels_this_combination] + \
                    C_gamma[index_1][channels_this_combination] + \
                    C_gamma[index_2][channels_this_combination]
                C = np.sum(C, axis=0)

                if _padding1 == 0:
                    for feature_count in range(num_features_this_dilation):
                        feature_index = feature_index_start + feature_count
                        _bias = biases[feature_index]

                        ppv = 0
                        last_val = 0
                        max_stretch = 0.0
                        mean_index = 0
                        mean = 0

                        for j in range(C.shape[0]):
                            if C[j] > _bias:
                                ppv += 1
                                mean_index += j
                                mean += C[j] + _bias
                            elif C[j] < _bias:
                                stretch = j - last_val
                                if stretch > max_stretch:
                                    max_stretch = stretch
                                last_val = j
                        stretch = C.shape[0] - 1 - last_val
                        if stretch > max_stretch:
                            max_stretch = stretch

                        end = feature_index
                        features[example_index, end] = ppv / C.shape[0]
                        end = end + num_features
                        features[example_index, end] = max_stretch
                        end = end + num_features
                        features[example_index, end] = mean / ppv if ppv > 0 else 0
                        end = end + num_features
                        features[example_index, end] = mean_index / ppv if ppv > 0 else -1
                else:
                    _c = C[padding:-padding]

                    for feature_count in range(num_features_this_dilation):
                        feature_index = feature_index_start + feature_count
                        _bias = biases[feature_index]

                        ppv = 0
                        last_val = 0
                        max_stretch = 0.0
                        mean_index = 0
                        mean = 0

                        for j in range(_c.shape[0]):
                            if _c[j] > _bias:
                                ppv += 1
                                mean_index += j
                                mean += _c[j] + _bias
                            elif _c[j] < _bias:
                                stretch = j - last_val
                                if stretch > max_stretch:
                                    max_stretch = stretch
                                last_val = j
                        stretch = _c.shape[0] - 1 - last_val
                        if stretch > max_stretch:
                            max_stretch = stretch

                        end = feature_index
                        features[example_index, end] = ppv / _c.shape[0]
                        end = end + num_features
                        features[example_index, end] = max_stretch
                        end = end + num_features
                        features[example_index, end] = mean / ppv if ppv > 0 else 0
                        end = end + num_features
                        features[example_index, end] = mean_index / ppv if ppv > 0 else -1

                feature_index_start = feature_index_end

                combination_index += 1
                num_channels_start = num_channels_end

        # First order difference
        _X1 = X1[example_index]
        A1 = -_X1  # A = alpha * X = -X
        G1 = _X1 + _X1 + _X1  # G = gamma * X = 3X

        feature_index_start = 0

        combination_index = 0
        num_channels_start = 0

        for dilation_index in range(num_dilations1):

            _padding0 = dilation_index % 2

            dilation = dilations1[dilation_index]
            padding = ((9 - 1) * dilation) // 2

            num_features_this_dilation = num_features_per_dilation1[dilation_index]

            C_alpha = np.zeros((num_channels, input_length - 1), dtype=np.float32)
            C_alpha[:] = A1

            C_gamma = np.zeros((9, num_channels, input_length - 1), dtype=np.float32)
            C_gamma[9 // 2] = G1

            start = dilation
            end = input_length - padding

            for gamma_index in range(9 // 2):
                C_alpha[:, -end:] = C_alpha[:, -end:] + A1[:, :end]
                C_gamma[gamma_index, :, -end:] = G1[:, :end]

                end += dilation

            for gamma_index in range(9 // 2 + 1, 9):
                C_alpha[:, :-start] = C_alpha[:, :-start] + A1[:, start:]
                C_gamma[gamma_index, :, :-start] = G1[:, start:]

                start += dilation

            for kernel_index in range(num_kernels):

                feature_index_end = feature_index_start + num_features_this_dilation

                num_channels_this_combination = num_channels_per_combination[combination_index]

                num_channels_end = num_channels_start + num_channels_this_combination

                channels_this_combination = channel_indices[num_channels_start:num_channels_end]

                _padding1 = (_padding0 + kernel_index) % 2

                index_0, index_1, index_2 = indices[kernel_index]

                C = C_alpha[channels_this_combination] + \
                    C_gamma[index_0][channels_this_combination] + \
                    C_gamma[index_1][channels_this_combination] + \
                    C_gamma[index_2][channels_this_combination]
                C = np.sum(C, axis=0)

                if _padding1 == 0:
                    for feature_count in range(num_features_this_dilation):
                        feature_index = feature_index_start + feature_count
                        _bias = biases1[feature_index]

                        ppv = 0
                        last_val = 0
                        max_stretch = 0.0
                        mean_index = 0
                        mean = 0

                        for j in range(C.shape[0]):
                            if C[j] > _bias:
                                ppv += 1
                                mean_index += j
                                mean += C[j] + _bias
                            elif C[j] < _bias:
                                stretch = j - last_val
                                if stretch > max_stretch:
                                    max_stretch = stretch
                                last_val = j
                        stretch = C.shape[0] - 1 - last_val
                        if stretch > max_stretch:
                            max_stretch = stretch

                        end = feature_index + n_features_per_transform
                        features[example_index, end] = ppv / C.shape[0]
                        end = end + num_features
                        features[example_index, end] = max_stretch
                        end = end + num_features
                        features[example_index, end] = mean / ppv if ppv > 0 else 0
                        end = end + num_features
                        features[example_index, end] = mean_index / ppv if ppv > 0 else -1
                else:
                    _c = C[padding:-padding]

                    for feature_count in range(num_features_this_dilation):
                        feature_index = feature_index_start + feature_count
                        _bias = biases1[feature_index]

                        ppv = 0
                        last_val = 0
                        max_stretch = 0.0
                        mean_index = 0
                        mean = 0

                        for j in range(_c.shape[0]):
                            if _c[j] > _bias:
                                ppv += 1
                                mean_index += j
                                mean += _c[j] + _bias
                            elif _c[j] < _bias:
                                stretch = j - last_val
                                if stretch > max_stretch:
                                    max_stretch = stretch
                                last_val = j
                        stretch = _c.shape[0] - 1 - last_val
                        if stretch > max_stretch:
                            max_stretch = stretch

                        end = feature_index + n_features_per_transform
                        features[example_index, end] = ppv / _c.shape[0]
                        end = end + num_features
                        features[example_index, end] = max_stretch
                        end = end + num_features
                        features[example_index, end] = mean / ppv if ppv > 0 else 0
                        end = end + num_features
                        features[example_index, end] = mean_index / ppv if ppv > 0 else -1

                feature_index_start = feature_index_end

    return features


class MultiRocket:

    def __init__(
            self,
            num_features=50000,
            classifier="ridge",
            verbose=0
    ):
        self.name = "MultiRocket"

        self.base_parameters = None
        self.diff1_parameters = None

        self.n_features_per_kernel = 4
        self.num_features = num_features / 2  # 1 per transformation
        self.num_kernels = int(self.num_features / self.n_features_per_kernel)

        if verbose > 1:
            print('[{}] Creating {} with {} kernels'.format(self.name, self.name, self.num_kernels))

        self.clf = classifier
        self.classifier = None
        self.train_duration = 0
        self.test_duration = 0
        self.generate_kernel_duration = 0
        self.train_transforms_duration = 0
        self.test_transforms_duration = 0
        self.apply_kernel_on_train_duration = 0
        self.apply_kernel_on_test_duration = 0

        self.verbose = verbose

    def fit(self, x_train, y_train, predict_on_train=True):

        if self.verbose > 1:
            print('[{}] Training with training set of {}'.format(self.name, x_train.shape))
        if x_train.shape[2] < 10:
            # handling very short series (like PensDigit from the MTSC archive)
            # series have to be at least a length of 10 (including differencing)
            _x_train = np.zeros((x_train.shape[0], x_train.shape[1], 10), dtype=x_train.dtype)
            _x_train[:, :, :x_train.shape[2]] = x_train
            x_train = _x_train
            del _x_train

        self.generate_kernel_duration = 0
        self.apply_kernel_on_train_duration = 0
        self.train_transforms_duration = 0

        start_time = time.perf_counter()

        _start_time = time.perf_counter()
        xx = np.diff(x_train, 1)
        self.train_transforms_duration += time.perf_counter() - _start_time

        _start_time = time.perf_counter()
        self.base_parameters = fit(
            x_train,
            num_features=self.num_kernels
        )
        self.diff1_parameters = fit(
            xx,
            num_features=self.num_kernels
        )
        self.generate_kernel_duration += time.perf_counter() - _start_time

        _start_time = time.perf_counter()
        x_train_transform = transform(
            x_train, xx,
            self.base_parameters, self.diff1_parameters,
            self.n_features_per_kernel
        )
        self.apply_kernel_on_train_duration += time.perf_counter() - _start_time

        x_train_transform = np.nan_to_num(x_train_transform)

        elapsed_time = time.perf_counter() - start_time
        if self.verbose > 1:
            print('[{}] Kernels applied!, took {}s'.format(self.name, elapsed_time))
            print('[{}] Transformed Shape {}'.format(self.name, x_train_transform.shape))

        if self.verbose > 1:
            print('[{}] Training'.format(self.name))

        if self.clf.lower() == "ridge":
            self.classifier = make_pipeline(
                StandardScaler(),
                RidgeClassifierCV(
                    alphas=np.logspace(-3, 3, 10),
                )
            )
        else:
            self.classifier = LogisticRegression(
                num_features=x_train_transform.shape[1],
                max_epochs=3000,
            )
        _start_time = time.perf_counter()
        self.classifier.fit(x_train_transform, y_train)
        self.train_duration = time.perf_counter() - _start_time

        if self.verbose > 1:
            print('[{}] Training done!, took {:.3f}s'.format(self.name, self.train_duration))
        if predict_on_train:
            yhat = self.classifier.predict(x_train_transform)
        else:
            yhat = None

        return yhat

    def predict(self, x):
        if self.verbose > 1:
            print('[{}] Predicting'.format(self.name))

        self.apply_kernel_on_test_duration = 0
        self.test_transforms_duration = 0

        _start_time = time.perf_counter()
        xx = np.diff(x, 1)
        self.test_transforms_duration += time.perf_counter() - _start_time

        _start_time = time.perf_counter()
        x_transform = transform(
            x, xx,
            self.base_parameters, self.diff1_parameters,
            self.n_features_per_kernel
        )
        self.apply_kernel_on_test_duration += time.perf_counter() - _start_time

        x_transform = np.nan_to_num(x_transform)
        if self.verbose > 1:
            print('Kernels applied!, took {:.3f}s. Transformed shape: {}.'.format(self.apply_kernel_on_test_duration,
                                                                                  x_transform.shape))
        start_time = time.perf_counter()
        yhat = self.classifier.predict(x_transform)
        self.test_duration = time.perf_counter() - start_time
        if self.verbose > 1:
            print("[{}] Predicting completed, took {:.3f}s".format(self.name, self.test_duration))

        return yhat

## Construct models

### Construct CNN-LSTM model type

In [ ]:
# Create the LSTM model
model = lstm(window_size, data.shape[1], 1, 5)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_layer_names = False, dpi=90, show_layer_activations=True, rankdir='TB')

### Construct ResNet

In [ ]:
#Resnet
nb_classes = len(np.unique(np.concatenate((y_train_noisy, y_val), axis=0)))

In [ ]:
# transform the labels from integers to one hot vectors
enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train_noisy, y_val), axis=0).reshape(-1, 1))
y_train_res = enc.transform(y_train_noisy.reshape(-1, 1)).toarray()
y_val_res = enc.transform(y_val.reshape(-1, 1)).toarray()

In [ ]:
# save orignal y because later we will use binary
y_val_true = np.argmax(y_val_res, axis=1)

In [ ]:
input_shape = x_train.shape[1:]
verbose = True
output_directory = './drive/MyDrive/kursinis/Modeliai/'

classifier = Classifier_RESNET(output_directory, input_shape, nb_classes, verbose)

### Construct Regulat LSTM multi


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

### Construct FCN (Fully Convolutional Network)

In [ ]:
nb_classes = len(np.unique(np.concatenate((y_train_noisy, y_val), axis=0)))

In [ ]:
# Create the LSTM model
model = fcn((window_size, data.shape[1]), nb_classes)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

### Construct Inception

In [ ]:
nb_classes = len(np.unique(np.concatenate((y_train_noisy, y_val), axis=0)))

In [ ]:
# transform the labels from integers to one hot vectors
enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train_noisy, y_val), axis=0).reshape(-1, 1))
y_train_res = enc.transform(y_train_noisy.reshape(-1, 1)).toarray()
y_val_res = enc.transform(y_val.reshape(-1, 1)).toarray()

In [ ]:
# save orignal y because later we will use binary
y_val_true = np.argmax(y_val_res, axis=1)

In [ ]:
input_shape = x_train.shape[1:]
verbose = True
output_directory = './drive/MyDrive/kursinis/Modeliai/'

# classifier = Classifier_INCEPTION(output_directory, input_shape, nb_classes, verbose)

classifier = Classifier_INCEPTION(
    output_directory,
    input_shape,
    nb_classes,
    verbose,
    batch_size=32,
    nb_filters=32,
    depth=6,
    kernel_size=41)

### Construct MLSTM-FCN

In [ ]:
nb_classes = len(np.unique(np.concatenate((y_train_noisy, y_val), axis=0)))

In [ ]:
# Create the LSTM model
model = mlstm_fcn((window_size, data.shape[1]), nb_classes)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

### Construct MultiRocket

In [ ]:
import argparse
import os
import platform
import socket
import time
from datetime import datetime

import numba
import numpy as np
import pandas as pd
import psutil
import pytz
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sktime.datasets import load_from_tsfile_to_dataframe

pd.set_option('display.max_columns', 500)

itr = 0
num_features = 10000
save = True
num_threads = 0

In [ ]:
output_path = os.getcwd() + "/output/"
classifier_name = "MultiRocket_{}".format(num_features)

In [ ]:
nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

In [ ]:
print(nb_classes)

In [ ]:
X_train = x_train.astype(np.float64).copy()
X_test = x_test.astype(np.float64).copy()
X_val = x_val.astype(np.float64).copy()

In [ ]:
X_train = np.transpose(X_train, axes=(0, 2, 1))
X_test = np.transpose(X_test, axes=(0, 2, 1))
X_val = np.transpose(X_val, axes=(0, 2, 1))


X_train.shape, X_val.shape, X_test.shape

In [ ]:
start = time.perf_counter()

output_dir = "{}/multirocket/resample_{}/{}/".format(
            output_path,
            itr,
            classifier_name
        )

In [ ]:
classifier = MultiRocket(
            num_features=50000,
            classifier="logistic",
            verbose=2
        )

## Train model

### Train CNN-LSTM

#### Train regular

In [ ]:
import datetime
import tensorboard
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard

In [ ]:
checkpoint_filepath = './drive/MyDrive/kursinis/Modeliai/model_CNN-LSTM-GPT-Strong_300_mul_.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True)
modeL_plateau_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    factor=0.5,
    patience=20,
    min_lr=0.0001
)

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    update_freq="epoch",
)

In [ ]:
model.load_weights('./drive/MyDrive/kursinis/Modeliai/model_CNN-LSTM-GPT-Strong_300_mul_.07-1.38.h5')

In [ ]:
# Reshape the input data to add an extra dimension
x_train_noisy = np.expand_dims(x_train_noisy, axis=1)
x_train_noisy.shape

In [ ]:
x_train_noisy = np.squeeze(x_train_noisy, axis=1)
x_train_noisy.shape

In [ ]:
x_val = np.expand_dims(x_val, axis=1)
x_val.shape

In [ ]:
x_val = np.squeeze(x_val, axis=1)
x_val.shape

In [ ]:
x_test = np.expand_dims(x_test, axis=1)
x_test.shape

In [ ]:
x_test = np.squeeze(x_test, axis=1)
x_test.shape

In [ ]:
# Train the model
model.fit(
    x_train_noisy,
    y_train_noisy,
    validation_data=(x_val, y_val),
    epochs=3000,
    batch_size=32,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[model_checkpoint_callback, tensorboard_callback]
)

In [ ]:
%tensorboard --logdir logs

#### Train KF Split

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

In [ ]:
n_splits=5

# Initialize cross-validation splitter
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Load dataset
X, y = signals_array, labels_array  # Load your dataset here

In [ ]:
cv_accuracies = []
cv_precisions = []
cv_recalls = []
cv_f1_scores = []
all_reports = []

In [ ]:
# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    x_train_kf, x_val_kf = X[train_index], X[val_index]
    y_train_kf, y_val_kf = y[train_index], y[val_index]

    print(x_train_kf.shape)
    print(x_val_kf.shape)

    # Compute mean and standard deviation over the training set
    mu = np.mean(x_train_kf)
    va = np.std(x_train_kf)

    print(mu)
    print(va)

    x_train_kf = (x_train_kf - mu) / va
    x_val_kf = (x_val_kf - mu) / va

    print(np.mean(x_train_kf))
    print(np.std(x_train_kf))
    print(np.mean(x_val_kf))
    print(np.std(x_val_kf))

    x_train_kf = np.expand_dims(x_train_kf, axis=1)
    x_val_kf = np.expand_dims(x_val_kf, axis=1)

    class_weights = compute_class_weight(
                                            class_weight = "balanced",
                                            classes = np.unique(y_train_kf),
                                            y = y_train_kf
                                        )
    class_weights = dict(zip(np.unique(y_train_kf), class_weights))
    class_weights

    nb_classes = len(np.unique(np.concatenate((y_train_kf, y_val_kf), axis=0)))

    # Create the LSTM model
    model = lstm(window_size, data.shape[1], 1, nb_classes)

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Create and compile FCN model
    model.fit(
        x_train_kf,
        y_train_kf,
        validation_data=(x_val_kf, y_val_kf),
        epochs=2,
        batch_size=32,
        shuffle=True,
        class_weight=class_weights,
        callbacks=[model_checkpoint_callback, modeL_plateau_callback]
    )

    # Evaluate the model and calculate accuracy
    predictions = model.predict(x_val_kf)

    # Calculate accuracy
    accuracy = accuracy_score(y_val_kf, np.argmax(predictions, axis=1))

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(y_val_kf, np.argmax(predictions, axis=1), average='weighted')

    accuracy *= 100
    precision *= 100
    recall *= 100
    f1 *= 100

    # Generate classification report
    report = classification_report(y_val_kf, np.argmax(predictions, axis=1))

    # Append scores to lists
    all_reports.append(report)
    cv_accuracies.append(accuracy)
    cv_precisions.append(precision)
    cv_recalls.append(recall)
    cv_f1_scores.append(f1)

    # Print results for this fold
    print(f"Accuracy for fold {fold + 1}: {accuracy:.2f}%")
    print(f"Precision for fold {fold + 1}: {precision:.2f}%")
    print(f"Recall for fold {fold + 1}: {recall:.2f}%")
    print(f"F1Score for fold {fold + 1}: {f1:.2f}%")
    print(report)
    print()

In [ ]:
for fold in range(n_splits):
    print(f"Fold {fold + 1}:")
    print("Accuracy:", cv_accuracies[fold])
    print("Classification report:")
    print(all_reports[fold])
    print()

print("Average Accuracy:", np.mean(cv_accuracies))
print("Average Precision:", np.mean(cv_precisions))
print("Average Recall:", np.mean(cv_recalls))
print("Average F1-score:", np.mean(cv_f1_scores))

In [ ]:
# Calculate average accuracy and standard deviation
average_accuracy = np.mean(cv_accuracies)
std_dev_accuracy = np.std(cv_accuracies)

average_precision = np.mean(cv_precisions)
std_dev_precision = np.std(cv_precisions)

average_recall = np.mean(cv_recalls)
std_dev_recall = np.std(cv_recalls)

average_f1_scores = np.mean(cv_f1_scores)
std_dev_f1_scores = np.std(cv_f1_scores)

# Calculate error rate
error_rate_accuracy = std_dev_accuracy / np.sqrt(len(cv_accuracies))
error_rate_precision = std_dev_precision / np.sqrt(len(cv_precisions))
error_rate_recall = std_dev_recall / np.sqrt(len(cv_recalls))
error_rate_f1_score = std_dev_f1_scores / np.sqrt(len(cv_f1_scores))


# Print results
print(f"Average Accuracy: {average_accuracy:.2f}% +- {error_rate_accuracy:.2f}%")
print(f"Average Precision: {average_precision:.2f}% +- {error_rate_precision:.2f}%")
print(f"Average Recall: {average_recall:.2f}% +- {error_rate_recall:.2f}%")
print(f"Average F1 score: {average_f1_scores:.2f}% +- {error_rate_f1_score:.2f}%")

### Train ResNet

#### Train Regular

In [ ]:
classifier.fit(
    x_train_noisy,
    y_train_res,
    x_val,
    y_val_res,
    y_val_true,
    batch_size = 32,
    nb_epochs = 3000,
    shuffle=True,
    class_weights=class_weights)

#### Train KF split

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

In [ ]:
n_splits=5

# Initialize cross-validation splitter
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Load dataset
X, y = signals_array, labels_array  # Load your dataset here

In [ ]:
cv_accuracies = []
cv_precisions = []
cv_recalls = []
cv_f1_scores = []
all_reports = []

In [ ]:
# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    x_train_kf, x_val_kf = X[train_index], X[val_index]
    y_train_kf, y_val_kf = y[train_index], y[val_index]

    print(x_train_kf.shape)
    print(x_val_kf.shape)

    # Compute mean and standard deviation over the training set
    mu = np.mean(x_train_kf)
    va = np.std(x_train_kf)

    print(mu)
    print(va)

    x_train_kf = (x_train_kf - mu) / va
    x_val_kf = (x_val_kf - mu) / va

    print(np.mean(x_train_kf))
    print(np.std(x_train_kf))
    print(np.mean(x_val_kf))
    print(np.std(x_val_kf))

    class_weights = compute_class_weight(
                                            class_weight = "balanced",
                                            classes = np.unique(y_train_kf),
                                            y = y_train_kf
                                        )
    class_weights = dict(zip(np.unique(y_train_kf), class_weights))
    class_weights

    nb_classes = len(np.unique(np.concatenate((y_train_kf, y_val_kf), axis=0)))

    x_train_kf, y_train_kf = add_laplace_noise_to_array(x_train_kf, y_train_kf, 0.05, 12)

    print(x_train_kf.shape)
    print(y_train_kf.shape)

    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    enc.fit(np.concatenate((y_train_kf, y_val_kf), axis=0).reshape(-1, 1))
    y_train_res = enc.transform(y_train_kf.reshape(-1, 1)).toarray()
    y_val_res = enc.transform(y_val_kf.reshape(-1, 1)).toarray()

    # save orignal y because later we will use binary
    y_val_true = np.argmax(y_val_res, axis=1)

    input_shape = x_train_kf.shape[1:]
    verbose = True
    output_directory = './drive/MyDrive/kursinis/Modeliai/'

    classifier = Classifier_RESNET(output_directory, input_shape, nb_classes, verbose)

    classifier.fit(
        x_train_kf,
        y_train_res,
        x_val_kf,
        y_val_res,
        y_val_true,
        batch_size = 16,
        nb_epochs = 150,
        shuffle=True,
        class_weights=class_weights)

    # Evaluate the model and calculate accuracy
    predictions = classifier.model.predict(x_val_kf)

    # Calculate accuracy
    accuracy = accuracy_score(y_val_kf, np.argmax(predictions, axis=1))

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(y_val_kf, np.argmax(predictions, axis=1), average='weighted')

    accuracy *= 100
    precision *= 100
    recall *= 100
    f1 *= 100

    report = classification_report(y_val_kf, np.argmax(predictions, axis=1))

    # Append scores to lists
    all_reports.append(report)
    cv_accuracies.append(accuracy)
    cv_precisions.append(precision)
    cv_recalls.append(recall)
    cv_f1_scores.append(f1)

    # Print results for this fold
    print(f"Accuracy for fold {fold + 1}: {accuracy:.2f}%")
    print(f"Precision for fold {fold + 1}: {precision:.2f}%")
    print(f"Recall for fold {fold + 1}: {recall:.2f}%")
    print(f"F1Score for fold {fold + 1}: {f1:.2f}%")
    print(report)
    print()

In [ ]:
for fold in range(n_splits):
    print(f"Fold {fold + 1}:")
    print("Accuracy:", cv_accuracies[fold])
    print("Classification report:")
    print(all_reports[fold])
    print()

print("Average Accuracy:", np.mean(cv_accuracies))
print("Average Precision:", np.mean(cv_precisions))
print("Average Recall:", np.mean(cv_recalls))
print("Average F1-score:", np.mean(cv_f1_scores))

In [ ]:
# Calculate average accuracy and standard deviation
average_accuracy = np.mean(cv_accuracies)
std_dev_accuracy = np.std(cv_accuracies)

average_precision = np.mean(cv_precisions)
std_dev_precision = np.std(cv_precisions)

average_recall = np.mean(cv_recalls)
std_dev_recall = np.std(cv_recalls)

average_f1_scores = np.mean(cv_f1_scores)
std_dev_f1_scores = np.std(cv_f1_scores)

# Calculate error rate
error_rate_accuracy = std_dev_accuracy / np.sqrt(len(cv_accuracies))
error_rate_precision = std_dev_precision / np.sqrt(len(cv_precisions))
error_rate_recall = std_dev_recall / np.sqrt(len(cv_recalls))
error_rate_f1_score = std_dev_f1_scores / np.sqrt(len(cv_f1_scores))


# Print results
print(f"Average Accuracy: {average_accuracy:.2f}% +- {error_rate_accuracy:.2f}%")
print(f"Average Precision: {average_precision:.2f}% +- {error_rate_precision:.2f}%")
print(f"Average Recall: {average_recall:.2f}% +- {error_rate_recall:.2f}%")
print(f"Average F1 score: {average_f1_scores:.2f}% +- {error_rate_f1_score:.2f}%")

### Train FCN

In [ ]:
checkpoint_filepath = './drive/MyDrive/kursinis/Modeliai/model_FCN_.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True)

modeL_plateau_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    factor=0.5,
    patience=50,
    min_lr=0.0001
)

In [ ]:
model.fit(
    x_train_noisy,
    y_train_noisy,
    validation_data=(x_val, y_val),
    epochs=3000,
    batch_size=32,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[model_checkpoint_callback, modeL_plateau_callback]
)

#### Train KF Split

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

In [ ]:
n_splits=5

# Initialize cross-validation splitter
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Load dataset
X, y = signals_array, labels_array  # Load your dataset here

In [ ]:
cv_accuracies = []
cv_precisions = []
cv_recalls = []
cv_f1_scores = []
all_reports = []

In [ ]:
# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    x_train_kf, x_val_kf = X[train_index], X[val_index]
    y_train_kf, y_val_kf = y[train_index], y[val_index]

    print(x_train_kf.shape)
    print(x_val_kf.shape)

    # Compute mean and standard deviation over the training set
    mu = np.mean(x_train_kf)
    va = np.std(x_train_kf)

    print(mu)
    print(va)

    x_train_kf = (x_train_kf - mu) / va
    x_val_kf = (x_val_kf - mu) / va

    print(np.mean(x_train_kf))
    print(np.std(x_train_kf))
    print(np.mean(x_val_kf))
    print(np.std(x_val_kf))

    class_weights = compute_class_weight(
                                            class_weight = "balanced",
                                            classes = np.unique(y_train_kf),
                                            y = y_train_kf
                                        )
    class_weights = dict(zip(np.unique(y_train_kf), class_weights))
    class_weights

    nb_classes = len(np.unique(np.concatenate((y_train_kf, y_val_kf), axis=0)))

    # Create the LSTM model
    model = fcn((window_size, data.shape[1]), nb_classes)

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Create and compile FCN model
    model.fit(
        x_train_kf,
        y_train_kf,
        validation_data=(x_val_kf, y_val_kf),
        epochs=2,
        batch_size=32,
        shuffle=True,
        class_weight=class_weights,
        callbacks=[model_checkpoint_callback, modeL_plateau_callback]
    )

    # Evaluate the model and calculate accuracy
    predictions = model.predict(x_val_kf)

    # Calculate accuracy
    accuracy = accuracy_score(y_val_kf, np.argmax(predictions, axis=1))

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(y_val_kf, np.argmax(predictions, axis=1), average='weighted')

    accuracy *= 100
    precision *= 100
    recall *= 100
    f1 *= 100

    # Generate classification report
    report = classification_report(y_val_kf, np.argmax(predictions, axis=1))

    # Append scores to lists
    all_reports.append(report)
    cv_accuracies.append(accuracy)
    cv_precisions.append(precision)
    cv_recalls.append(recall)
    cv_f1_scores.append(f1)

    # Print results for this fold
    print(f"Accuracy for fold {fold + 1}: {accuracy:.2f}%")
    print(f"Precision for fold {fold + 1}: {precision:.2f}%")
    print(f"Recall for fold {fold + 1}: {recall:.2f}%")
    print(f"F1Score for fold {fold + 1}: {f1:.2f}%")
    print(report)
    print()

In [ ]:
for fold in range(n_splits):
    print(f"Fold {fold + 1}:")
    print("Accuracy:", cv_accuracies[fold])
    print("Classification report:")
    print(all_reports[fold])
    print()

print("Average Accuracy:", np.mean(cv_accuracies))
print("Average Precision:", np.mean(cv_precisions))
print("Average Recall:", np.mean(cv_recalls))
print("Average F1-score:", np.mean(cv_f1_scores))

In [ ]:
# Calculate average accuracy and standard deviation
average_accuracy = np.mean(cv_accuracies)
std_dev_accuracy = np.std(cv_accuracies)

average_precision = np.mean(cv_precisions)
std_dev_precision = np.std(cv_precisions)

average_recall = np.mean(cv_recalls)
std_dev_recall = np.std(cv_recalls)

average_f1_scores = np.mean(cv_f1_scores)
std_dev_f1_scores = np.std(cv_f1_scores)

# Calculate error rate
error_rate_accuracy = std_dev_accuracy / np.sqrt(len(cv_accuracies))
error_rate_precision = std_dev_precision / np.sqrt(len(cv_precisions))
error_rate_recall = std_dev_recall / np.sqrt(len(cv_recalls))
error_rate_f1_score = std_dev_f1_scores / np.sqrt(len(cv_f1_scores))


# Print results
print(f"Average Accuracy: {average_accuracy:.2f}% +- {error_rate_accuracy:.2f}%")
print(f"Average Precision: {average_precision:.2f}% +- {error_rate_precision:.2f}%")
print(f"Average Recall: {average_recall:.2f}% +- {error_rate_recall:.2f}%")
print(f"Average F1 score: {average_f1_scores:.2f}% +- {error_rate_f1_score:.2f}%")

### Train Inception

In [ ]:
classifier.fit(
    x_train_noisy,
    y_train_res,
    x_val,
    y_val_res,
    y_val_true,
    batch_size = 32,
    nb_epochs = 3000,
    shuffle=True,
    class_weights=class_weights)

#### Train KF Split

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

In [ ]:
n_splits=5

# Initialize cross-validation splitter
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Load dataset
X, y = signals_array, labels_array  # Load your dataset here

In [ ]:
cv_accuracies = []
cv_precisions = []
cv_recalls = []
cv_f1_scores = []
all_reports = []

In [ ]:
# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    x_train_kf, x_val_kf = X[train_index], X[val_index]
    y_train_kf, y_val_kf = y[train_index], y[val_index]

    print(x_train_kf.shape)
    print(x_val_kf.shape)

    # Compute mean and standard deviation over the training set
    mu = np.mean(x_train_kf)
    va = np.std(x_train_kf)

    print(mu)
    print(va)

    x_train_kf = (x_train_kf - mu) / va
    x_val_kf = (x_val_kf - mu) / va

    print(np.mean(x_train_kf))
    print(np.std(x_train_kf))
    print(np.mean(x_val_kf))
    print(np.std(x_val_kf))

    class_weights = compute_class_weight(
                                            class_weight = "balanced",
                                            classes = np.unique(y_train_kf),
                                            y = y_train_kf
                                        )
    class_weights = dict(zip(np.unique(y_train_kf), class_weights))
    class_weights

    nb_classes = len(np.unique(np.concatenate((y_train_kf, y_val_kf), axis=0)))

    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    enc.fit(np.concatenate((y_train_kf, y_val_kf), axis=0).reshape(-1, 1))
    y_train_res = enc.transform(y_train_kf.reshape(-1, 1)).toarray()
    y_val_res = enc.transform(y_val_kf.reshape(-1, 1)).toarray()

    # save orignal y because later we will use binary
    y_val_true = np.argmax(y_val_res, axis=1)

    input_shape = x_train_kf.shape[1:]
    verbose = True
    output_directory = './drive/MyDrive/kursinis/Modeliai/'

    # classifier = Classifier_INCEPTION(output_directory, input_shape, nb_classes, verbose)

    classifier = Classifier_INCEPTION(
        output_directory,
        input_shape,
        nb_classes,
        verbose,
        batch_size=32,
        nb_filters=32,
        depth=6,
        kernel_size=41)

    classifier.fit(
        x_train_kf,
        y_train_res,
        x_val_kf,
        y_val_res,
        y_val_true,
        batch_size = 32,
        nb_epochs = 1500,
        shuffle=True,
        class_weights=class_weights)

    # Evaluate the model and calculate accuracy
    predictions = classifier.model.predict(x_val_kf)

    # Calculate accuracy
    accuracy = accuracy_score(y_val_kf, np.argmax(predictions, axis=1))

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(y_val_kf, np.argmax(predictions, axis=1), average='weighted')

    accuracy *= 100
    precision *= 100
    recall *= 100
    f1 *= 100

    report = classification_report(y_val_kf, np.argmax(predictions, axis=1))

    # Append scores to lists
    all_reports.append(report)
    cv_accuracies.append(accuracy)
    cv_precisions.append(precision)
    cv_recalls.append(recall)
    cv_f1_scores.append(f1)

    # Print results for this fold
    print(f"Accuracy for fold {fold + 1}: {accuracy:.2f}%")
    print(f"Precision for fold {fold + 1}: {precision:.2f}%")
    print(f"Recall for fold {fold + 1}: {recall:.2f}%")
    print(f"F1Score for fold {fold + 1}: {f1:.2f}%")
    print(report)
    print()

In [ ]:
for fold in range(n_splits):
    print(f"Fold {fold + 1}:")
    print("Accuracy:", cv_accuracies[fold])
    print("Classification report:")
    print(all_reports[fold])
    print()

print("Average Accuracy:", np.mean(cv_accuracies))
print("Average Precision:", np.mean(cv_precisions))
print("Average Recall:", np.mean(cv_recalls))
print("Average F1-score:", np.mean(cv_f1_scores))

In [ ]:
# Calculate average accuracy and standard deviation
average_accuracy = np.mean(cv_accuracies)
std_dev_accuracy = np.std(cv_accuracies)

average_precision = np.mean(cv_precisions)
std_dev_precision = np.std(cv_precisions)

average_recall = np.mean(cv_recalls)
std_dev_recall = np.std(cv_recalls)

average_f1_scores = np.mean(cv_f1_scores)
std_dev_f1_scores = np.std(cv_f1_scores)

# Calculate error rate
error_rate_accuracy = std_dev_accuracy / np.sqrt(len(cv_accuracies))
error_rate_precision = std_dev_precision / np.sqrt(len(cv_precisions))
error_rate_recall = std_dev_recall / np.sqrt(len(cv_recalls))
error_rate_f1_score = std_dev_f1_scores / np.sqrt(len(cv_f1_scores))


# Print results
print(f"Average Accuracy: {average_accuracy:.2f}% +- {error_rate_accuracy:.2f}%")
print(f"Average Precision: {average_precision:.2f}% +- {error_rate_precision:.2f}%")
print(f"Average Recall: {average_recall:.2f}% +- {error_rate_recall:.2f}%")
print(f"Average F1 score: {average_f1_scores:.2f}% +- {error_rate_f1_score:.2f}%")

### Train MLSTM-FCN

In [ ]:
checkpoint_filepath = './drive/MyDrive/kursinis/Modeliai/model_MLSTM-FCN_.{epoch:02d}-{val_loss:.2f}.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    verbose=1,
    save_best_only=True)

modeL_plateau_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    factor=0.5,
    patience=50,
    min_lr=0.0001
)

In [ ]:
model.fit(
    x_train_noisy,
    y_train_noisy,
    validation_data=(x_val, y_val),
    epochs=3000,
    batch_size=32,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[model_checkpoint_callback, modeL_plateau_callback]
)

### Train MultiRocket

In [ ]:
yhat_train = classifier.fit(
            X_train, y_train,
            predict_on_train=True
        )

In [ ]:
if yhat_train is not None:
    train_acc = accuracy_score(y_train, yhat_train)
else:
    train_acc = -1

## Evaluate model

### Evaluate CNN-LSTM

#### Softmax

In [ ]:
model.load_weights('./drive/MyDrive/kursinis/Modeliai/model_LSTM_.2452-0.96.h5')

In [ ]:
# Evaluate the model
model.evaluate(x_test, y_test, verbose=1)

In [ ]:
# Evaluate the model and calculate accuracy
predictions = model.predict(x_test)

In [ ]:
print(predictions.shape)
print(x_test.shape)

# Calculate accuracy for each label
rounded_predictions = np.argmax(predictions, axis=1)

# Calculate accuracy for each label
correct = np.sum(rounded_predictions == y_test)

total_samples = len(y_test)
accuracy = correct / total_samples * 100
print(f"Overall Accuracy: {accuracy:.2f}%")

In [ ]:
from sklearn.metrics import classification_report
unique_labels, counts = np.unique(labels_array, return_counts=True)
print("Window size:", window_size)
print("Step size:", step_size)
print()
print("Unique Labels:", unique_labels)
print("Class counts:", counts)
print("Original data:", signals_array.shape)
print()
print("Train data (No aug):", x_train.shape)
print("Val data (No aug):", x_val.shape)
print("Test data (No aug):", x_test.shape)
print()
print("Train data augmented:", x_train_noisy.shape)
print()
print(f"Test set accuracy: {accuracy:.2f}%")
print()
print(classification_report(y_test, rounded_predictions))

### Evaluate ResNet

#### Softmax

In [ ]:
#Load best model
model_path = classifier.output_directory + 'best_model.498-1.19.h5'
model = keras.models.load_model(model_path)

start_time = time.time()
y_pred = model.predict(x_test)

In [ ]:
start_time = time.time()
y_pred = classifier.model.predict(x_test)

In [ ]:
print(y_pred.shape)
print(x_test.shape)

# Calculate accuracy for each label
rounded_predictions = np.argmax(y_pred, axis=1)

# Calculate accuracy for each label
correct = np.sum(rounded_predictions == y_test)

total_samples = len(y_test)
accuracy = correct / total_samples * 100
print(f"Overall Accuracy: {accuracy:.2f}%")

In [ ]:
from sklearn.metrics import classification_report
unique_labels, counts = np.unique(labels_array, return_counts=True)
print("Window size:", window_size)
print("Step size:", step_size)
print()
print("Unique Labels:", unique_labels)
print("Class counts:", counts)
print("Original data:", signals_array.shape)
print()
print("Train data (No aug):", x_train.shape)
print("Val data (No aug):", x_val.shape)
print("Test data (No aug):", x_test.shape)
print()
# print("Train data augmented:", x_train_noisy.shape)
print()
print(f"Test set accuracy: {accuracy:.2f}%")
print()
print(classification_report(y_test, rounded_predictions))

### Evaluate FCN

In [ ]:
model.load_weights('./drive/MyDrive/kursinis/Modeliai/model_FCN_.479-0.73.h5')

In [ ]:
# Evaluate the model
model.evaluate(x_test, y_test, verbose=1)

In [ ]:
# Evaluate the model and calculate accuracy
predictions = model.predict(x_test)

In [ ]:
print(predictions.shape)
print(x_test.shape)

# Calculate accuracy for each label
rounded_predictions = np.argmax(predictions, axis=1)

# Calculate accuracy for each label
correct = np.sum(rounded_predictions == y_test)

total_samples = len(y_test)
accuracy = correct / total_samples * 100
print(f"Overall Accuracy: {accuracy:.2f}%")

In [ ]:
from sklearn.metrics import classification_report
unique_labels, counts = np.unique(labels_array, return_counts=True)
print("Window size:", window_size)
print("Step size:", step_size)
print()
print("Unique Labels:", unique_labels)
print("Class counts:", counts)
print("Original data:", signals_array.shape)
print()
print("Train data (No aug):", x_train.shape)
print("Val data (No aug):", x_val.shape)
print("Test data (No aug):", x_test.shape)
print()
print("Train data augmented:", x_train_noisy.shape)
print()
print(f"Test set accuracy: {accuracy:.2f}%")
print()
print(classification_report(y_test, rounded_predictions))

### Evaluate MultiRocket

In [ ]:
yhat_test = classifier.predict(X_test)
test_acc = accuracy_score(y_test, yhat_test)

In [ ]:
yhat_val = classifier.predict(X_val)
val_acc = accuracy_score(y_val, yhat_val)

In [ ]:
print(train_acc)

print(val_acc)

print(test_acc)

### Calculate Means

In [ ]:
from collections import defaultdict
import numpy as np

In [ ]:
# Define dictionary to store precision, recall, and F1-score for each class across folds
class_metrics = defaultdict(lambda: {'precision': [], 'recall': [], 'f1-score': []})

input_string = """
Fold 1:
Accuracy: 90.2439024390244
Classification report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      0.67      0.80         6
           2       0.75      0.75      0.75         8
           3       0.88      1.00      0.93         7
           4       1.00      1.00      1.00        14

    accuracy                           0.90        41
   macro avg       0.90      0.88      0.88        41
weighted avg       0.91      0.90      0.90        41


Fold 2:
Accuracy: 87.8048780487805
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       0.75      0.86      0.80         7
           2       0.75      0.43      0.55         7
           3       0.78      1.00      0.88         7
           4       1.00      1.00      1.00        13

    accuracy                           0.88        41
   macro avg       0.86      0.86      0.84        41
weighted avg       0.88      0.88      0.87        41


Fold 3:
Accuracy: 90.0
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.83      0.71      0.77         7
           2       0.67      0.86      0.75         7
           3       1.00      0.86      0.92         7
           4       1.00      1.00      1.00        13

    accuracy                           0.90        40
   macro avg       0.90      0.89      0.89        40
weighted avg       0.91      0.90      0.90        40


Fold 4:
Accuracy: 95.0
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.78      1.00      0.88         7
           2       1.00      0.71      0.83         7
           3       1.00      1.00      1.00         7
           4       1.00      1.00      1.00        13

    accuracy                           0.95        40
   macro avg       0.96      0.94      0.94        40
weighted avg       0.96      0.95      0.95        40


Fold 5:
Accuracy: 92.5
Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       0.83      0.71      0.77         7
           2       0.75      0.86      0.80         7
           3       1.00      1.00      1.00         7
           4       1.00      1.00      1.00        13

    accuracy                           0.93        40
   macro avg       0.92      0.91      0.91        40
weighted avg       0.93      0.93      0.92        40


"""

folds = input_string.strip().split("Fold")[1:6]

for report in folds:
    lines = report.strip().split('\n')
    class_lines = lines[5:-4]  # Extract lines containing precision, recall, and F1-score for each class
    for line in class_lines:
        print(line)
        class_id, precision, recall, f1_score, _ = line.split()
        class_id = int(class_id)
        precision = float(precision)
        recall = float(recall)
        f1_score = float(f1_score)
        class_metrics[class_id]['precision'].append(precision)
        class_metrics[class_id]['recall'].append(recall)
        class_metrics[class_id]['f1-score'].append(f1_score)

# Calculate mean precision, recall, and F1-score for each class
mean_metrics = {}
for class_id, metrics in class_metrics.items():
    mean_precision = np.mean(metrics['precision'])
    mean_recall = np.mean(metrics['recall'])
    mean_f1_score = np.mean(metrics['f1-score'])
    mean_metrics[class_id] = {'precision': mean_precision, 'recall': mean_recall, 'f1-score': mean_f1_score}

# Print the results
for class_id, metrics in mean_metrics.items():
    print(f"Class {class_id}:")
    print(f"  Mean Precision: {metrics['precision']:.2f}")
    print(f"  Mean Recall: {metrics['recall']:.2f}")
    print(f"  Mean F1-score: {metrics['f1-score']:.2f}")



In [ ]:
# Print the results in LaTeX format
print("Precision:", " & ".join([f"{metrics['precision'] * 100:.0f}" for class_id, metrics in mean_metrics.items()]), "\\\\")
print("Recall:", " & ".join([f"{metrics['recall'] * 100:.0f}" for class_id, metrics in mean_metrics.items()]), "\\\\")
print("F1:", " & ".join([f"{metrics['f1-score'] * 100:.0f}" for class_id, metrics in mean_metrics.items()]), "\\\\")

In [ ]:
import os

# Install de_DE
!/usr/share/locales/install-language-pack de_DE
!dpkg-reconfigure locales

# Restart Python process to pick up the new locales
os.kill(os.getpid(), 9)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import locale
from statsmodels.stats.proportion import proportions_ztest

# Set locale to use comma as decimal point
locale.setlocale(locale.LC_NUMERIC, "de_DE")

# Data from the table (mean accuracy and standard deviation)
data = {
    'Be augmentacijų': [64.32, 1.58, 88.12, 1.48, 91.11, 1.09, 89.09, 1.54],
    'Laplasas (std/100)': [62.35, 4.00, 89.10, 1.35, 91.61, 1.08, 92.06, 1.32],
    'Laplasas (std/50)': [61.40, 1.71, 89.61, 1.91, 91.09, 0.90, 90.57, 1.12],
    'Laplasas (std/20)': [59.91, 1.68, 91.10, 1.11, 91.60, 0.86, 92.56, 0.72],
    # Add data for other augmentation techniques...
    'Dreifuojantis Gausas (std/100)': [61.89, 3.04, 91.09, 1.82, 90.10, 0.71, 91.07, 1.68],
    'Dreifuojantis Gausas (std/50)': [64.38, 1.54, 88.10, 2.87, 92.09, 1.47, 91.57, 1.14],
    'Dreifuojantis Gausas (std/20)': [61.85, 1.61, 92.59, 1.86, 90.12, 0.94, 89.48, 1.08],
    'Tolygusis (std/100)': [62.87, 2.20, 89.13, 1.91, 90.07, 1.44, 91.11, 1.09],
    'Tolygusis (std/50)': [59.93, 1.30, 89.62, 1.27, 91.09, 1.14, 90.59, 0.47],
    'Tolygusis (std/20)': [59.94, 2.71, 89.12, 1.49, 89.09, 1.54, 89.09, 2.09],
    'Laplasas ir Dreifuojantis Gausas': [63.39, 2.56, 92.60, 1.54, 91.07, 0.92, 90.59, 0.85],
    'Laplasas ir Tolygusis': [62.90, 1.97, 89.11, 1.51, 91.10, 0.87, 92.57, 0.71],
    'Dydžio deformacija': [59.38, 2.13, 88.11, 1.93, 90.56, 2.19, 90.10, 1.41],
    'TimeVAE': [75.73, 0.88, 77.17, 2.75, 90.05, 2.93, 88.57, 2.90],
    # Add data for other augmentation techniques...
}

# Models and augmentation techniques
models = ['CNN-LSTM', 'FCN', 'ResNet', 'InceptionTime']

# Augmentation techniques
augmentation_techniques = list(data.keys())

# Base augmentation (Be augmentacijų) accuracy
base_accuracies = data['Be augmentacijų'][::2]

print(base_accuracies)

colors = ['#ff5234', '#1f77b4', '#ff7f0e', '#2ca02c']

# Plot bar plot with error bars
plt.figure(figsize=(20, 15))  # Increase figure size

plt.rcParams['axes.formatter.use_locale'] = True
locale._override_localeconv["decimal_point"]= ','
plt.rcdefaults()

# Tell matplotlib to use the locale we set above
plt.rcParams['axes.formatter.use_locale'] = True

# Width of the bars
bar_width = 0.2

# Spacing between groups
spacing = 0.02  # Increase spacing

# Position for each group
positions = np.arange(len(augmentation_techniques))

# Plot each model's accuracy
for i, model in enumerate(models):
    accuracies = [data[technique][i*2] if data[technique][i*2] is not None else np.nan for technique in augmentation_techniques]
    errors = [data[technique][i*2 + 1] if data[technique][i*2] is not None else np.nan for technique in augmentation_techniques]
    bars = plt.bar(positions + i * (bar_width + spacing), accuracies, yerr=errors, width=bar_width, label=model, color = colors[i])
    # Add percentages on bars (vertically, centered)
    for j, (bar, acc, err) in enumerate(zip(bars, accuracies, errors)):
        base_acc = base_accuracies[i]
        if not np.isnan(acc):
            acc_str = locale.format_string("%.2f", acc, grouping=True)
            err_str = locale.format_string("%.2f", err, grouping=True)
            if acc > base_acc:

                if (model == 'CNN-LSTM' and augmentation_techniques[j] == 'TimeVAE') or (model == 'FCN' and (augmentation_techniques[j] == 'Dreifuojantis Gausas (std/100)' or augmentation_techniques[j] == 'Dreifuojantis Gausas (std/20)' or augmentation_techniques[j] == 'Laplasas ir Dreifuojantis Gausas')) or (model == 'InceptionTime' and augmentation_techniques[j] == 'Laplasas ir Tolygusis'):
                    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 15, f'{acc_str} ± -{err_str}*', ha='center', va='bottom', rotation=90, fontsize=16, weight='bold')
                else:
                    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 15, f'{acc_str} ± {err_str}', ha='center', va='bottom', rotation=90, fontsize=16)


            else:
                plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 15, f'{acc_str} ± {err_str}', ha='center', va='bottom', rotation=90, fontsize=16)


# Set x-axis labels
plt.xticks(positions + (len(models)) * (bar_width + spacing) / 2, augmentation_techniques, rotation=45, fontsize=16, ha='right')

# Set y-axis ticks every 10%
plt.ylim(40,100)
plt.yticks(np.arange(40, 101, 10))

# Add horizontal lines for each 10%
for y in range(40, 101, 10):
    plt.axhline(y=y, color='gray', linestyle='--', linewidth=0.5)

# Add labels and legend
plt.xlabel('Duomenų augmentacijos metodai', fontsize=16)
plt.ylabel('Klasifikavimo tikslumas, %', fontsize=16)
# plt.title('Mean Classification Accuracy with Error Bars')
plt.legend(loc='upper right', bbox_to_anchor=(1.07, 1.04), fontsize=15)  # Move legend to upper right corner with bbox_to_anchor

# Show plot
plt.tight_layout()
plt.show()

In [ ]:
# Create data
group1 = [90.24, 92.68, 82.5, 90.0, 90.0]
group2 = [95.12, 92.68, 90.0, 95.0, 87.5]

# conduct the Wilcoxon-Signed Rank Test
stats.wilcoxon(group1, group2, alternative='less')